In [1]:
import cProfile
import os
import statistics as st
import plotly.graph_objects as go
import random as rd
import time
import pandas as pd
import math
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut
import Games.mnk as mnk
import Games.Carcassonne.Carcassonne as carc
import Games.carcassonne_oldtry as csn_old
#import Games.carcassonne_older as csn
import Agents.random as arand
import Agents.vanilla_mcts as mcts
import Agents.siea_mcts as siea_mcts
import Agents.siea_mcts2 as siea2
import Agents.rea_mcts as rea_mcts
import Agents.mcts_rave as rave_mcts
import Agents.one_step_lookahead as osla
import Agents.mcts_solver as mcts_solver
import Agents.minimax as minimax
import Games.function_optimisation as fo
import Utilities.logs_management as lm
import Games.chess_64 as chess_64
import chess as chess
import chess.svg
import matplotlib.pyplot as plt
import shutil
import itertools
from IPython.display import display
import ast
from IPython.display import Image as DImage
import datetime
import multiprocessing as mp
from plotly.subplots import make_subplots
from IPython.display import HTML
import scipy.stats as scst
from PIL import Image

#cProfile.run("wins =  random_games(10000, base_gs)")
#ut.run()

def t(alpha, gl):
    return scst.t.ppf(1-(alpha/2), gl)
#https://stackoverflow.com/questions/67340028/how-to-use-t-ppf-which-are-the-arguments
#https://stackoverflow.com/questions/40294844/calculate-t-student-test-for-the-next-table-in-python
#https://www.geeksforgeeks.org/python-students-t-distribution-in-statistics/
#https://itl.nist.gov/div898/handbook/eda/section3/eda352.htm
#https://docs.tibco.com/data-science/GUID-8E64444A-5F5B-49F9-A43A-74B13647AFB4.html#:~:text=The%20larger%20the%20variation%2C%20the%20less%20reliable%20the,sample%20size%20is%20large%2C%20say%20n%3D100%20or%20more.

c:\Users\Fredasus\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Fredasus\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Fredasus\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Fredasu

In [16]:
#View tree on each iteration - STEP 1

#parameters
game_idx = 4
agent_idx = 0
agent_rollouts = 1
#puzzle_idx = 66 #matein3
puzzle_idx = 62
#max_iterations = 0
max_iterations = 0
iterations_logs_step = 10
max_time = 15

#database
games = ["chess", "ttt", "ttt_setup", "carcassonne", "carcassonne_1", "carcassonne_discard","chess_puzzle", "ttt_setup", "carcassonne_sequence", "carc_sequence_1p","fop"]
game = games[game_idx]
print("Game:", game)
agent_types = ["mcts", "rave_mcts", "siea_mcts", "mcts_solver", "siea_mcts2", "rea_mcts","minimax","random"]
agent_type = agent_types[agent_idx]
print("Agent:", agent_type)

#agent definition
if agent_type == "mcts": #0
    mcts_agent = mcts.MCTS_Player(rollouts=agent_rollouts, max_time = max_time) #max_iterations = max_iterations,
elif agent_type == "rave_mcts": #1
    mcts_agent = rave_mcts.MCTS_RAVE(max_iterations = max_iterations, rollouts=agent_rollouts)
elif agent_type == "siea_mcts": #2
    mcts_agent = siea_mcts.SIEA_MCTS_Player(max_iterations = max_iterations, rollouts=agent_rollouts, logs=True)
elif agent_type == "mcts_solver": #3
    mcts_agent = mcts_solver.MCTS_Solver(max_iterations = max_iterations, rollouts=agent_rollouts)
elif agent_type == "siea_mcts2": #4
    mcts_agent = siea2.SIEA_MCTS_Player2(max_iterations = max_iterations, rollouts=agent_rollouts)
elif agent_type == "rea_mcts": #5
    mcts_agent = rea_mcts.SIEA_MCTS_Player(max_iterations = max_iterations#, logs=True
    , rollouts=agent_rollouts
    , parallel_evolution=True
    , no_terminal_no_parent=True
    , re_evaluation=True)
elif agent_type == "minimax":  #6
    mcts_agent = minimax.Minimax(logs=True, max_depth=3)
elif agent_type == "random":  #7
    mcts_agent = arand.RandomPlayer(logs=True)

#game definition
if game == "chess": #0
    game_state = chess_64.GameState()
    game_state.set_initial_state()
elif game == "ttt": #1
    game_state = mnk.GameState(3,3,3)
    game_state.set_initial_state()
elif game == "ttt_setup": #2
    game_state = mnk.GameState(3,3,3)
    game_state.set_board(board_items = {(1,1):0, 
                                    (0,0):0, 
                                    (2,2):1, 
                                    (0,1):1,
                                    #(1,0):0
                                    }, player_turn = 0)
elif game == "carcassonne": #3
    game_state = carc.CarcassonneState()
    game_state.set_initial_state()
elif game == "carcassonne_1": #4
    game_state = carc.CarcassonneState(initial_tile_quantities=[1 for _ in range(24)],players=1, initial_meeples=[2,2], set_tile_sequence=True, set_tile_sequence_seed=2)
    game_state.set_initial_state()
elif game == "carcassonne_discard": #5
    game_state = carc.CarcassonneState(initial_tile_quantities=[1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0], players=1)
    game_state.set_initial_state()
elif game == "carcassonne_sequence": #6
    game_state = carc.CarcassonneState(initial_tile_quantities=[1 for _ in range(24)], set_tile_sequence=True, set_tile_sequence_seed=1, initial_meeples=[2,2])
    game_state.set_initial_state()
elif game == "carc_sequence_1p": #7
    game_state = carc.CarcassonneState(initial_tile_quantities=[1 for _ in range(24)], set_tile_sequence=True, set_tile_sequence_seed=1, initial_meeples=[2,2], players = 1)
    game_state.set_initial_state()
elif game == "chess_puzzle":    #8
    game_state = chess_64.GameState()
    #Load lichess dataset
    lichess_db = pd.read_csv("Datasets/mates_processed_lichess_db_puzzle.csv")
    puzzle_row = lichess_db.iloc[puzzle_idx]
    game_state.set_puzzle_lichess_db(puzzle_row)
    print("Move to look for:", puzzle_row["Moves"].split(" ")[1])
    print("Moves to look for:", puzzle_row["Moves"])
    print(puzzle_row["Themes"])
elif game == "fop": #9
    game_state = fo.GameState(function_index=1, n_players=1, splits = 3)
    game_state.set_initial_state()
    
#game_state.view_game_state()
print(len(game_state.available_actions))

Game: carcassonne_1
Agent: mcts
20


In [11]:
#debug minimax
game_state.sample_random_event(enforced_outcome=13)
print(len(game_state.available_actions))
print(game_state.next_tile_index, len(game_state.available_actions))
#mcts_agent.heuristic_function = eu.heuristic_function_ttt
#while not game_state.is_terminal:
returned_action = mcts_agent.choose_action(game_state)#, verbose=True)
#game_state.make_action(returned_action)
print(returned_action)
print(mcts_agent.current_fm)
#print(game_state.Scores)

13 12
TileIndex: 13, (X,Y): (0,1), Rotation: 180, Meeple Location: City, Location Index: 0
3172286


In [17]:
#Initialize mcts agent
max_iterations = 1
if max_iterations == 0:
    random_action = mcts_agent.choose_action(game_state)
    print("N available_moves:", len(game_state.available_actions))
    #print("Root node:", mcts_agent.view_mcts_tree())
    #game_state.view_game_state()
    print("Player:",mcts_agent.player)
else:
    mcts_agent.choose_action(game_state)
    print("Current decision node: ", mcts_agent.recommendation_policy())
    print("Iterations: ", mcts_agent.current_iterations)
    print("FM calls: ", mcts_agent.current_fm)
    #print(mcts_agent.view_mcts_tree())
    print(mcts_agent.view_action_stats())
    #print(mcts_agent.view_proven_tree())
    if "ea" in agent_type:
        print("Evolution fm calls: ", mcts_agent.evolution_fm_calls)
    print("Node count: ", mcts_agent.nodes_count)
#game_state.board

Current decision node:  TileIndex: 15, (X,Y): (-1,0), Rotation: 270, Meeple Location: Monastery, Location Index: 0
Iterations:  3002
FM calls:  69046

0:Decision_node, edge:None, visits:3002, avg_rwd:0.0738, children:20,from:20
1:Decision_node, edge:TileIndex: 15, (X,Y): (0,-1), Rotation: 0, Meeple Location: Monastery, Location Index: 0, visits:169, avg_rwd:0.0925, children:18,from:18, tree_policy_formula:0.4
2:Decision_node, edge:TileIndex: 15, (X,Y): (0,-1), Rotation: 90, Meeple Location: Farm, Location Index: 0, visits:137, avg_rwd:0.058, children:14,from:14, tree_policy_formula:0.4
3:Decision_node, edge:TileIndex: 15, (X,Y): (0,-1), Rotation: 270, Meeple Location: Monastery, Location Index: 0, visits:168, avg_rwd:0.0911, children:18,from:18, tree_policy_formula:0.4
4:Decision_node, edge:TileIndex: 15, (X,Y): (0,-1), Rotation: 270, Meeple Location: Farm, Location Index: 0, visits:139, avg_rwd:0.0604, children:14,from:14, tree_policy_formula:0.4
5:Decision_node, edge:TileIndex: 15, (

In [20]:
#View tree on each iteration - STEP 2 - Iterate

#By iteration
iterations = 1
def run_iteration(iterations):
    for it in range(iterations):
        mcts_agent.iteration()
#run_iteration(iterations)
#cProfile.run("run_iteration(iterations)")

#By_choose_action
iterations = 1100
def run_choose_action():
    mcts_agent.max_iterations = iterations
    mcts_agent.choose_action(game_state)
run_choose_action()

print("Iterations: ", mcts_agent.current_iterations)
print("FM calls: ", mcts_agent.current_fm)
print("Recommended action: ", mcts_agent.recommendation_policy())
print(mcts_agent.view_mcts_tree())
#print(mcts_agent.view_action_stats())
#print(mcts_agent.view_proven_tree())
#DImage("Games/Carcassonne/pygame_images/"+ str(game_state.next_tile_index)+".png")

Iterations:  1100
FM calls:  12100
Recommended action:  [['0.667', '1.000']]t0

0:Decision_node, edge:None, visits:1100, avg_rwd:0.816, children:3,from:3, tree_policy:None
--1:Decision_node, edge:[['0.667', '1.000']]t0, visits:831, avg_rwd:0.864, children:3,from:3, tree_policy:0.994
----4:Decision_node, edge:[['0.667', '0.778']]t0, visits:42, avg_rwd:0.476, children:3,from:3, tree_policy:1.04
------10:Decision_node, edge:[['0.704', '0.741']]t0, visits:13, avg_rwd:0.462, children:3,from:3, tree_policy:1.22
--------79:Decision_node, edge:[['0.716', '0.728']]t0, visits:3, avg_rwd:0.333, children:2,from:3, tree_policy:1.64
----------561:Decision_node, edge:[['0.720', '0.724']]t0, visits:1, avg_rwd:1, children:0,from:3, tree_policy:2.48
----------562:Decision_node, edge:[['0.724', '0.728']]t0, visits:1, avg_rwd:0, children:0,from:3, tree_policy:1.48
--------217:Decision_node, edge:[['0.728', '0.741']]t0, visits:5, avg_rwd:0.6, children:3,from:3, tree_policy:1.61
----------314:Decision_node,

In [21]:
#VOI
def value_of_information(node):
    assert node.parent is not None
    if len(list(node.parent.children.values())) < 2:
        return 0
    sorted_siblings = sorted(list(node.parent.children.values()), key=lambda x: x.average_reward(), reverse=True)
    alpha_node = sorted_siblings[0]
    beta_node = sorted_siblings[1]
    if node.average_reward() == alpha_node.average_reward(): #if this node is the best node
        return (beta_node.average_reward()/(node.visits+1))*math.exp(-2*node.visits*(alpha_node.average_reward() - beta_node.average_reward())**2)
    else:
        return ((1-alpha_node.average_reward())/(node.visits+1))*math.exp(-2*node.visits*(alpha_node.average_reward() - node.average_reward())**2)
    

for edge_action,child_node in mcts_agent.root_node.children.items():
    print("")#Child node: ", child_node)
    print("VOI: ", value_of_information(child_node))
    print("Visits: ", child_node.visits)
    print("Average reward: ", child_node.average_reward())


VOI:  4.4550012267396966e-23
Visits:  831
Average reward:  0.8640192539109507

VOI:  1.0822088762409626e-07
Visits:  167
Average reward:  0.7005988023952096

VOI:  5.532535093549676e-09
Visits:  102
Average reward:  0.6176470588235294


CARC DEVALUE ANALYSIS

In [30]:
#PARAMETERS
#logs_path = os.path.join("Outputs", "Carcassonne_analysis", "meeples 7_games 1000_seed 0_fulltiles True_deterministic False")# This exp is ok
logs_path = os.path.join("Outputs", "Carcassonne_analysis", "meeples 1_games 1000_seed 0_fulltiles False_deterministic False")# This exp is ok
#logs_path = os.path.join("Outputs", "Carcassonne_analysis", "FINAL_meeples7_games100_seed0_fulltilesTrue_deterministicFalse_players22nd_turn_after_tile13")
colors = ["#5B8C5A","#56638A","#EC7316","#e265ef","#c27dd1","#d52e87","#a8d0b9","#ad8dfd"]
replaces = {"(1,0)":"Top", "(0,1)":"Right", "(-1,0)":"Bottom", "(0,-1)":"Left"}

In [22]:
#Count actions

#paramters:
full_tiles = False
deterministic = False
game_seed = 0
meeples = 3
players = 1
devalue_games = 100
second_turn = False
enforced_fist_turn_tile = 0

if full_tiles:
    initial_tile_quantities = [1,3,1,1,2,3,2,2,2,3,1,3,2,5,3,2,4,3,3,4,4,9,8,1]
else:
    initial_tile_quantities = [1 for _ in range(24)]

game_state = carc.CarcassonneState(name = "Carcassonne",
                                                initial_tile_quantities = initial_tile_quantities,
                                                set_tile_sequence= deterministic,
                                                set_tile_sequence_seed=game_seed,
                                                initial_meeples = [meeples, meeples],
                                                players = players)
game_state.set_initial_state()

if second_turn:
    game_state.sample_random_event(event_type = "Draw random tile", enforced_outcome = enforced_fist_turn_tile)

    if enforced_fist_turn_tile == 13:
        #find and play action that closes the initial city
        for action in game_state.available_actions:
            fclone_state = game_state.duplicate()
            fclone_state.make_action(action)
            if fclone_state.Scores[0] == 4:
                print("winning action", action)
                break
    else:
        for action in game_state.available_actions:
            fclone_state = game_state.duplicate()
            fclone_state.make_action(action)
            if fclone_state.Scores[2] == 0:
                print("winning action", action)
                break
    
    game_state = fclone_state   

#Initialise agent: 
action_counts = []
for tile_index, tile_quantity in enumerate(game_state.TileQuantities):
    if tile_quantity > 0 and tile_index != 0: #REMOVE TILE INDEX CONDITION, ONLY HERE FOR THE SECOND MOVE IF NO CITIES ARE AVAILABLE
        game_state.sample_random_event(event_type = "Draw random tile", enforced_outcome = tile_index)
        action_counts.append(len(game_state.available_actions))
#print mean
print("mean", st.mean(action_counts), "+-", st.stdev(action_counts))
#print total
print("total", sum(action_counts))
#print max score
print(game_state.max_score(verbose=True))

mean 20.545454545454547 +- 13.312104311945621
total 452
City points:  52
Road points:  20
Monastery points:  18
Closed cities:  5
Farm points:  45
Max score: 135
135


In [15]:
#Aux data because of error not including the full action. Now added in the code, but still need to run this to fix the old data
full_tiles = False
game_seed = 0

#calculations
if full_tiles:
    initial_tile_quantities = [1,3,1,1,2,3,2,2,2,3,1,3,2,5,3,2,4,3,3,4,4,9,8,1]
else:
    initial_tile_quantities = [1 for _ in range(24)]

#Initialise carcassonne game
game_state = carc.CarcassonneState(name = "dummy",
                                                initial_tile_quantities = initial_tile_quantities,
                                                set_tile_sequence= False,
                                                set_tile_sequence_seed=game_seed,
                                                initial_meeples = [1, 1],
                                                players = 2)
game_state.set_initial_state()

actions_dict = {"action":[],"First_move_tile_index":[], "First_move_action_index":[], "tile_location":[], "tile_rotation":[], "tile_probability":[]}
for tile_index, tile_quantity in enumerate(game_state.TileQuantities):
    if tile_quantity > 0:
        game_state.sample_random_event(event_type = "Draw random tile", enforced_outcome = tile_index)
        for action_index, available_action in enumerate(game_state.available_actions):
            actions_dict["action"].append(str(available_action))
            actions_dict["First_move_tile_index"].append(tile_index)
            actions_dict["First_move_action_index"].append(action_index)
            actions_dict["tile_location"].append(str(available_action).split("(X,Y): ")[1].split(", Rotation: ")[0])
            actions_dict["tile_rotation"].append(str(available_action).split("Rotation: ")[1].split(",")[0])
            actions_dict["tile_probability"].append(game_state.TileQuantities[tile_index]+1/sum(game_state.TileQuantities))
actions_df = pd.DataFrame(actions_dict)
#print(actions_df.head())
cc = lm.dump_data(pd.DataFrame(actions_dict), file_path = logs_path, file_name="actions_dict.csv")

In [13]:
#COLLECT LOGS - Only ran the first time

collective_logs = pd.DataFrame()
collective_action_logs = pd.DataFrame()
#print(os.listdir(output_path))
for tile_index in os.listdir(logs_path):
    #if path is a folder
    if os.path.isdir(os.path.join(logs_path, tile_index)):
        #print(tile_index)
        for action_index in os.listdir(os.path.join(logs_path, tile_index)):
            #print(action_index)
            for meeple_type in os.listdir(os.path.join(logs_path, tile_index, action_index)):
                #print(meeple_type)
                game_log = lm.read_csv(file_path = os.path.join(logs_path, tile_index, action_index, meeple_type, "logs_by_game.csv"))
                action_log = lm.read_csv(file_path = os.path.join(logs_path, tile_index, action_index, meeple_type, "logs_by_action.csv"))
                game_def_data = lm.read_csv(file_path = os.path.join(logs_path, tile_index, action_index, meeple_type, "game_definition_data.csv"))
                game_log["First_move_meeple_type"] = [meeple_type for _ in range(len(game_log))]
                game_log["First_move_tile_index"] = [tile_index.split("_")[1] for _ in range(len(game_log))]
                game_log["First_move_action_index"] = [action_index.split("_")[2] for _ in range(len(game_log))]
                game_log["Max_possible_score"] = [game_def_data["max_possible_score"][0] for _ in range(len(game_log))]
                game_log.replace("Monastery","Cloister")
                collective_logs = pd.concat([collective_logs, game_log], ignore_index = True)
                action_data = pd.merge(action_log[["game_index","Turn","Player_turn","Meeples_on_farms_p1","Meeples_on_farms_p2","Total_played_meeples_p1","Total_played_meeples_p2"]], 
                                       game_log[["game_index","Score_p1","Score_p2","First_move_meeple_type","Winner"]], on = ["game_index"])
                collective_action_logs = pd.concat([collective_action_logs, action_data], ignore_index = True)
cc = lm.dump_data(collective_logs, file_path = logs_path, file_name = "collective_logs.csv")
ca = lm.dump_data(collective_action_logs, file_path = logs_path, file_name = "collective_action_logs.csv")

In [ ]:
#PLOT SCORE DIFF by type of move

#define variables
#weighted = False
x_column = "First_move_meeple_type"
y_columns = ["Score_difference"]#["Score_p1", "Score_p2"]
y_columns2 = ["Score_p1", "Score_p2"]

#Create figure
fig = make_subplots(rows=1
                        ,cols=1
                        ,shared_xaxes=True
                        #,vertical_spacing=0.04
                        #,row_heights=[1/2 for _ in range(2)]
                        #,subplot_titles = ["Score difference", "Score"]
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Move type"
                        ,y_title='Score'
                        #,print_grid=True
                        )

data = lm.read_csv(file_path = os.path.join(logs_path ,"collective_logs.csv"))
data = data.replace("Monastery","Cloister")
data["Score_difference"] = data["Score_p1"] - data["Score_p2"]



for y_i, y_column in enumerate(y_columns):
    fig.add_trace(go.Box(x=data[x_column]
                        ,y=data[y_column]
                        ,name="Score"
                        , width=0.8
                        #,showlegend=True
                        ,legendgroup=y_i
                        , marker={"color":colors[y_i]}
                        )
                    ,row=1
                    ,col=1)
"""
for y_i, y_column in enumerate(y_columns2):
    fig.add_trace(go.Box(x=data[x_column]
                        ,y=data[y_column]
                        ,name=str(y_column.replace("_", " "))
                        #, width=0.8
                        ,showlegend=True
                        ,legendgroup=y_i + len(y_columns)
                        , marker={"color":colors[y_i + len(y_columns)]}
                        )
                    ,row=2
                    ,col=1)
"""

fig.update_layout(margin=dict(l=60, r=10, t=10, b=80)
                        ,width=500
                        ,height=500
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.16
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    ,tick0 = 0
                    , gridcolor="grey"
                    , gridwidth=1
                    , dtick=10
                    ,showgrid=True
                    ,zerolinecolor = "black"
                    ,zerolinewidth = 1
                    )
fig.write_image(os.path.join(logs_path, "Carcassonne_devalues_by_claimed_feature_t1.png"))
fig.show()

In [ ]:
#Triple dataset comparison
#PLOT SCORE DIFF by type of move
exp_path = os.path.join("Outputs", "Carcassonne_analysis")
dataset_paths = [
   # os.path.join(exp_path, "meeples 7_games 1000_seed 0_fulltiles True_deterministic False"),
    os.path.join(exp_path, "FINAL_meeples7_games100_seed0_fulltilesTrue_deterministicFalse_players22nd_turn_after_tile13"),
    os.path.join(exp_path, "FINAL_meeples7_games100_seed0_fulltilesTrue_deterministicFalse_players22nd_turn_after_tile0")]
dataset_names = [#"First turn", 
                 "B1 : With a completed city", "B2 : With a city less likely to be completed"]
substract_points = [#[0,0],
                    [4,0],[0,0]]
player_perspective = [#0,
                      1,1] #0 means from p1 perspective, 1 from p2 perspective



#define variables
#weighted = False
x_column = "First_move_meeple_type"
y_columns = ["Score_difference"]

#Create figure
fig = make_subplots(rows=1
                        ,cols=1
                        #,shared_xaxes=True
                        #,vertical_spacing=0.04
                        #,row_heights=[1/2 for _ in range(2)]
                        #,subplot_titles = ["Score difference", "Score"]
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Move type"
                        ,y_title='Score difference'
                        #,print_grid=True
                        )

data = [lm.read_csv(file_path = os.path.join(dataset_path ,"collective_logs.csv")) for dataset_path in dataset_paths]
data = [d.replace("Monastery","Cloister") for d in data]
for di, d in enumerate(data):
    if player_perspective[di] == 0:
        d["Score_difference"] = (d["Score_p1"] - substract_points[di][0]) - (d["Score_p2"] - substract_points[di][1])
    else:
        d["Score_difference"] = (d["Score_p2"] - substract_points[di][1]) - (d["Score_p1"] - substract_points[di][0])


for di, d in enumerate(data):
    for y_i, y_column in enumerate(y_columns):
        fig.add_trace(go.Box(x=d[x_column]
                            ,y=d[y_column]
                            ,name=dataset_names[di]
                            #, width=0.8
                            ,showlegend=True
                            ,legendgroup=di
                            , marker={"color":colors[di]}
                            )
                        ,row=1
                        ,col=1)
"""
for y_i, y_column in enumerate(y_columns2):
    fig.add_trace(go.Box(x=data[x_column]
                        ,y=data[y_column]
                        ,name=str(y_column.replace("_", " "))
                        #, width=0.8
                        ,showlegend=True
                        ,legendgroup=y_i + len(y_columns)
                        , marker={"color":colors[y_i + len(y_columns)]}
                        )
                    ,row=2
                    ,col=1)
"""


fig.update_layout(margin=dict(l=60, r=10, t=10, b=80)
                        ,width=500
                        ,height=500
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.16
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    ,tick0 = 0
                    , gridcolor="grey"
                    , gridwidth=1
                    , dtick=10
                    ,showgrid=True
                    ,zerolinecolor = "black"
                    ,zerolinewidth = 1
                    )
fig.write_image(os.path.join(exp_path, "Carcassonne_devalues_by_claimed_feature_t2.png"))
fig.show()

In [ ]:
#Scores by meeples on farms by turn
x_column = "Turn"
y_columns = ["Meeples_on_farms_p1","Total_non_farm_meeples","Total_played_meeples_p1"]
y_names = ["Meeples on farms", "Meeples on non-farm features", "Meeples on any feature"]

data = lm.read_csv(file_path = os.path.join(logs_path ,"collective_action_logs.csv"))
data["Total_non_farm_meeples"] = data["Total_played_meeples_p1"] - data["Meeples_on_farms_p1"]
data = data.replace("Monastery","Cloister")
data["Score_difference"] = data["Score_p1"] - data["Score_p2"]
data["Meeples_on_farms_difference"] = data["Meeples_on_farms_p1"] - data["Meeples_on_farms_p2"]
data["Played_meeples_difference"] = data["Total_played_meeples_p1"] - data["Total_played_meeples_p2"]
#filter
data = data[data["Player_turn"] == 0]

fig = make_subplots(rows=1
                        ,cols=len(y_columns)
                        ,shared_yaxes=True
                        ,horizontal_spacing=0.02
                        #,vertical_spacing=0.04
                        #,row_heights=[1/2 for _ in range(2)]
                        #,subplot_titles = y_columns
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        #,x_title=
                        ,y_title='Turn'
                        #,print_grid=True
                        )
z_mins = []
z_maxs = []
for y_i, y_column in enumerate(y_columns):
    data1 = data[["Turn", y_column,"Score_difference"]]
    grouped_data1 = data1.groupby(["Turn", y_column]).mean()

    lm.dump_data(grouped_data1, file_path = logs_path, file_name = "collective_action_logs_grouped_" + y_column +".csv")
    grouped_data1 = lm.read_csv(file_path = os.path.join(logs_path ,"collective_action_logs_grouped_" + y_column +".csv"))

    turns = sorted(list(set(grouped_data1["Turn"])))
    y_uniques = sorted(list(set(grouped_data1[y_column])))
    print(y_column, y_uniques)
    z = []
    for turn in turns:
        z_row = []
        for y_unique in y_uniques:
            subdata = grouped_data1[(grouped_data1["Turn"] == turn) & (grouped_data1[y_column] == y_unique)]
            if len(subdata) > 0:
                z_row.append(subdata["Score_difference"].mean())
            else:
                z_row.append(None)
        z.append(z_row)
    z_mins.append(min([min([zz for zz in z_row if zz is not None]) for z_row in z]))
    z_maxs.append(max([max([zz for zz in z_row if zz is not None]) for z_row in z]))
    #z_maxs.append(max([max(z_row) for z_row in z]))
    #print z shape
    print("z shape", np.array(z).shape)
    #Create figure
    fig.add_trace(go.Heatmap(y=[str(t) for t in turns]
                        ,x=[str(y_unique) for y_unique in y_uniques]
                        ,z=z
                        ,name="Final score, where the player tries to maximise"  #######ATTENTION TO THIS ONE
                        #, width=0.8
                        ,showlegend=y_i==0
                        ,legendgroup=1
                        #, marker={"color":colors[di]}
                        )
                    ,row=1
                    ,col=y_i+1)
    
    fig.update_xaxes(title_text=y_names[y_i], row=1, col=y_i+1)
    #fig.update_yaxes(title_text="Turn", row=1, col=y_i+1) 

# Update zmin and zmax for all heatmaps in the figure
zmin = min(z_mins)
zmax = max(z_maxs)
fig.update_traces(zmin=zmin, zmax=zmax, selector=dict(type='heatmap'))
fig.update_layout(margin=dict(l=60, r=10, t=10, b=80)
                        ,width=1000
                        ,height=500
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        #,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.16
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    #,tick0 = 0
                    #, gridcolor="grey"
                    #, gridwidth=1
                    #, dtick=10
                    #,showgrid=True
                    ,zerolinecolor = "black"
                    ,zerolinewidth = 1
                    )


fig.write_image(os.path.join(logs_path, "Carcassonne_scores_turn_meeple.png"))
fig.show()


In [ ]:
#Scores by meeples on farms by turn for p2
x_column = "Turn"
y_columns = ["Meeples_on_farms_p2","Total_non_farm_meeples","Total_played_meeples_p2"]
y_names = ["Meeples on farms", "Meeples on non-farm features", "Meeples on any feature"]

data = lm.read_csv(file_path = os.path.join(logs_path ,"collective_action_logs.csv"))
data["Total_non_farm_meeples"] = data["Total_played_meeples_p2"] - data["Meeples_on_farms_p2"]
data = data.replace("Monastery","Cloister")
data["Score_difference"] = data["Score_p2"] - data["Score_p1"]
data["Meeples_on_farms_difference"] = data["Meeples_on_farms_p2"] - data["Meeples_on_farms_p1"]
data["Played_meeples_difference"] = data["Total_played_meeples_p2"] - data["Total_played_meeples_p1"]
#filter
data = data[data["Player_turn"] == 1]

fig = make_subplots(rows=1
                        ,cols=len(y_columns)
                        ,shared_yaxes=True
                        ,horizontal_spacing=0.04
                        #,vertical_spacing=0.04
                        #,row_heights=[1/2 for _ in range(2)]
                        #,subplot_titles = y_columns
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        #,x_title=
                        ,y_title='Turn'
                        #,print_grid=True
                        )
z_mins = []
z_maxs = []
for y_i, y_column in enumerate(y_columns):
    data1 = data[["Turn", y_column,"Score_difference"]]
    grouped_data1 = data1.groupby(["Turn", y_column]).mean()

    lm.dump_data(grouped_data1, file_path = logs_path, file_name = "collective_action_logs_grouped_" + y_column +".csv")
    grouped_data1 = lm.read_csv(file_path = os.path.join(logs_path ,"collective_action_logs_grouped_" + y_column +".csv"))

    turns = sorted(list(set(grouped_data1["Turn"])))
    y_uniques = sorted(list(set(grouped_data1[y_column])))
    print(y_column, y_uniques)
    z = []
    for turn in turns:
        z_row = []
        for y_unique in y_uniques:
            subdata = grouped_data1[(grouped_data1["Turn"] == turn) & (grouped_data1[y_column] == y_unique)]
            if len(subdata) > 0:
                z_row.append(subdata["Score_difference"].mean())
            else:
                z_row.append(None)
        z.append(z_row)
    z_mins.append(min([min([zz for zz in z_row if zz is not None]) for z_row in z]))
    z_maxs.append(max([max([zz for zz in z_row if zz is not None]) for z_row in z]))
    #z_maxs.append(max([max(z_row) for z_row in z]))
    #print z shape
    print("z shape", np.array(z).shape)
    #Create figure
    fig.add_trace(go.Heatmap(y=[str(t) for t in turns]
                        ,x=[str(y_unique) for y_unique in y_uniques]
                        ,z=z
                        ,name="Score difference (player 2 - player 1), where player 2 tries to maximise"  #######ATTENTION TO THIS ONE
                        #, width=0.8
                        ,showlegend=y_i==0
                        ,legendgroup=1
                        #, marker={"color":colors[di]}
                        )
                    ,row=1
                    ,col=y_i+1)
    
    fig.update_xaxes(title_text=y_names[y_i], row=1, col=y_i+1)
    #fig.update_yaxes(title_text="Turn", row=1, col=y_i+1) 

# Update zmin and zmax for all heatmaps in the figure
zmin = min(z_mins)
zmax = max(z_maxs)
fig.update_traces(zmin=zmin, zmax=zmax, selector=dict(type='heatmap'))
fig.update_layout(margin=dict(l=60, r=10, t=10, b=80)
                        ,width=1000
                        ,height=500
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        #,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.16
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    #,tick0 = 0
                    #, gridcolor="grey"
                    #, gridwidth=1
                    #, dtick=10
                    #,showgrid=True
                    ,zerolinecolor = "black"
                    ,zerolinewidth = 1
                    )


fig.write_image(os.path.join(logs_path, "Carcassonne_scores_turn_meeple.png"))
fig.show()


In [ ]:
#PLOT SCOREs by type of move

#define variables
#weighted = False
x_column = "First_move_meeple_type"
y_columns = ["Score_p1", "Score_p2"]

#Create figure
fig = make_subplots(rows=1
                        ,cols=1
                        #,shared_xaxes=True
                        #,vertical_spacing=0.04
                        #,row_heights=[1/len(plot_identifiers) for _ in range(len(plot_identifiers))]
                        #,subplot_titles = [str(i) + " starting meeples" for i in plot_identifiers]
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Move type"
                        ,y_title='Score difference'
                        #,print_grid=True
                        )

data = lm.read_csv(file_path = os.path.join(logs_path ,"collective_logs.csv"))
data = data.replace("Monastery","Cloister")
data["Score"] = data["Score_p1"] - data["Score_p2"]



for y_i, y_column in enumerate(y_columns):
    fig.add_trace(go.Box(x=data[x_column]
                        ,y=data[y_column]
                        ,name=str(y_column.split("_")[1])
                        #, width=0.8
                        ,showlegend=True
                        ,legendgroup=y_i
                        , marker={"color":colors[y_i]}
                        )
                    ,row=1
                    ,col=1)

fig.update_layout(margin=dict(l=70, r=10, t=30, b=50)
                        ,width=700
                        ,height=400
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.075
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    ,tick0 = 0
                    , gridcolor="grey"
                    , gridwidth=1
                    , dtick=10
                    ,showgrid=True
                    )
fig.write_image(os.path.join(logs_path, "scores_by_meeple_type.png"))
fig.show()

In [ ]:
#Plot scores by type of move and location

#define variables
x_column = "First_move_meeple_type"
y_columns = ["Score_difference"]#["Score_p1", "Score_p2"]
plot_identifier_column = "tile_location"

data = lm.read_csv(file_path = os.path.join(logs_path ,"collective_logs.csv"))
data["Score_difference"] = data["Score_p1"] - data["Score_p2"]
actions_data = lm.read_csv(file_path = os.path.join(logs_path ,"actions_dict.csv"))
data = pd.merge(data, actions_data, on = ["First_move_tile_index", "First_move_action_index"])

#Create figure
plot_identifiers = data[plot_identifier_column].unique()
rows = len(plot_identifiers)
fig = make_subplots(rows=1
                        ,cols=1
                        #,shared_xaxes=True
                        #,vertical_spacing=0.04
                        #,row_heights=[1/rows for _ in range(rows)]
                        #,subplot_titles = plot_identifiers
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Move type"
                        ,y_title='Score difference'
                        #,print_grid=True
                        )

for group, plot_identifier in enumerate(data[plot_identifier_column].unique()):
    data_subset = data[data[plot_identifier_column] == plot_identifier]
    for y_i, y_column in enumerate(y_columns):
        fig.add_trace(go.Box(x=data_subset[x_column]
                            ,y=data_subset[y_column]
                            ,name=(replaces[plot_identifier] if plot_identifier in replaces else plot_identifier)
                            #, width=0.8
                            ,showlegend=y_i==0
                            ,legendgroup=group
                            , marker={"color":colors[group]}
                            )
                        ,row=1
                        ,col=1)

fig.update_layout(margin=dict(l=70, r=10, t=30, b=100)
                        ,width=600
                        ,height=400
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.24
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    ,tick0 = 0
                    , gridcolor="grey"
                    , gridwidth=0.1
                    , dtick=10
                    ,showgrid=True
                    )
fig.write_image(os.path.join(logs_path, "score_difference_by_meeple_type_and_location.png"))
fig.show()

In [ ]:
#Plot scores by type of move and location by tile index

#define variables
x_column = "First_move_meeple_type"
y_columns = ["Score_difference"]#["Score_p1", "Score_p2"]
plot_identifier_column = "First_move_tile_index"

data = lm.read_csv(file_path = os.path.join(logs_path ,"collective_logs.csv"))
data["Score_difference"] = data["Score_p1"] - data["Score_p2"]
actions_data = lm.read_csv(file_path = os.path.join(logs_path ,"actions_dict.csv"))
data = pd.merge(data, actions_data, on = ["First_move_tile_index", "First_move_action_index"])

#Create figure
plot_identifiers = data[plot_identifier_column].unique()
colors = eu.generate_color_palette(len(plot_identifiers))
rows = len(plot_identifiers)
fig = make_subplots(rows=1
                        ,cols=1
                        #,shared_xaxes=True
                        #,vertical_spacing=0.04
                        #,row_heights=[1/rows for _ in range(rows)]
                        #,subplot_titles = plot_identifiers
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Move type"
                        ,y_title='Score difference'
                        #,print_grid=True
                        )

for group, plot_identifier in enumerate(plot_identifiers):
    data_subset = data[data[plot_identifier_column] == plot_identifier]
    for y_i, y_column in enumerate(y_columns):
        fig.add_trace(go.Box(x=data_subset[x_column]
                            ,y=data_subset[y_column]
                            ,name="Tile index: " + str(plot_identifier)
                            #, width=0.8
                            ,showlegend=y_i==0
                            ,legendgroup=group
                            , marker={"color":colors[group]}
                            )
                        ,row=1
                        ,col=1)

fig.update_layout(margin=dict(l=70, r=10, t=30, b=100)
                        ,width=1200
                        ,height=400
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,boxmode='group'
                        ,legend=dict(
                            title = ""
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.24
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    ,tick0 = 0
                    , gridcolor="grey"
                    , gridwidth=0.1
                    , dtick=10
                    ,showgrid=True
                    )
fig.write_image(os.path.join(logs_path, "score_difference_by_meeple_type_and_tile_index.png"))
fig.show()

-----CARC SINGLE PLAYER

In [3]:
#parameters
#especific_experiment_path = "FINAL_10000"#"FINAL_Carcassonne_extra_games"
especific_experiment_path = "Combined_10000"
#especific_experiment_path = "FINAL_15_SEC"
experiment_path = os.path.join("Outputs","Carcassonne_1p",especific_experiment_path)
data_order = [
        #"MCTS_c0_5",
        "MCTS_c0.5",
        "MCTS_c0.5_15",
        #"MCTS_c1",
        #"MCTS_c1_4142",
        "MCTS_c1.4142135623730951",
        "MCTS_c1.4142135623730951_15",
        #"MCTS_c2",
        "MCTS_c3",
        "MCTS_c3_15",
        #"EA_MCTS_its2570",
        #"EA_MCTS_its5000",
        "EA_MCTS",
        "EA_MCTS_15",
        #"SIEA_MCTS_its2570",
        #"SIEA_MCTS_its5000",
        "SIEA_MCTS",
        "SIEA_MCTS_15",
        #"EA_MCTS2_its5000",
        #"EA_MCTS2_its7430",
        "Minimax_1_s",
        "Minimax_1_vs",
        "Minimax_2_s",
        "Minimax_2_vs",
        "Random",
    ]
data_order = data_order[::-1]
name_mappings = {"avg_leaf_node_depth":"Average leaf node depth","pg_game_turn":"Turn", "Total_played_meeples_p1":"Total meeples played", "Meeples_on_farms_p1":"Meeples on farms", "Meeples_on_cities_p1":"Meeples on cities", "Meeples_on_roads_p1":"Meeples on roads", "Meeples_on_cloisters_p1":"Meeples on cloisters", "Meeples_on_monasteries_p1":"Meeples on monasteries", "Meeples_on_non_farm_features_p1":"Meeples on non-farm features", "Meeples_on_any_feature_p1":"Meeples on any feature", "Score_p1":"Score", "Score_difference":"Score difference", "First_move_meeple_type":"Initial move type", "First_move_tile_index":"Initial action tile index", "First_move_action_index":"Initial action index", "tile_location":"Tile location", "tile_rotation":"Tile rotation", "tile_probability":"Tile probability", "Winner":"Winner", "Max_possible_score":"Max possible score","MCTS_c0_5":"MCTS C=0.5", "MCTS_c0.5":"MCTS C=0.5", "MCTS_c1":"MCTS C=1", "MCTS_c1_4142":"MCTS C=\u221A\u03052\u0305", "MCTS_c2":"MCTS C=2", "MCTS_c3":"MCTS C=3", "EA_MCTS_its2570":"EA-MCTS 2570", "EA_MCTS_its5000":"EA-MCTS", "SIEA_MCTS_its2570":"SIEA-MCTS 2570", "SIEA_MCTS_its5000":"SIEA-MCTS", "EA_MCTS2_its5000":"EA-MCTS2", "EA_MCTS2_its7430":"EA-MCTS2","max_visits":"Most visited node visit count", "Virtual_score_p1":"Virtual score","Meeples_p1":"Available meeples","Player_0":"MCTS variant","SIEA_MCTS":"SIEA-MCTS","EA_MCTS":"EA-MCTS", "MCTS_c1.4142135623730951":"MCTS C=\u221A\u03052\u0305", "Minimax_1_vs":"Expectimax-1-vs", "Minimax_2_vs":"Expectimax-2-vs", "Minimax_1_s":"Expectimax-1-s", "Minimax_2_s":"Expectimax-2-s", "Random":"Random", "forward_model_calls":"Forward model calls","MCTS_c0.5_15":"MCTS C=0.5", "MCTS_c1.4142135623730951_15":"MCTS C=\u221A\u03052\u0305", "MCTS_c3_15":"MCTS C=3", "EA_MCTS_15":"EA-MCTS", "SIEA_MCTS_15":"SIEA-MCTS","current_iterations":"Iterations"}
carc_variants = {(1,False):"Carc<sub>1,s</sub>", (1,True):"Carc<sub>1,d</sub>", (3,False):"Carc<sub>3,s</sub>", (3,True):"Carc<sub>3,d</sub>", (False,1):"Carc<sub>1,s</sub>", (True,1):"Carc<sub>1,d</sub>", (False,3):"Carc<sub>3,s</sub>", (True,3):"Carc<sub>3,d</sub>"}
line_styles = ['solid', 'dash', 'dot', 'dashdot', 'longdash', 'longdashdot', 'solid', 'dash', 'dot', 'dashdot']
marker_symbols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#losing_iterations_agents = ["EA_MCTS_its2570", "SIEA_MCTS_its2570", "EA_MCTS_its5000", "SIEA_MCTS_its5000"]
pattern_shapes = ["x","+","/","\\","|","-","o","O",".","*"]

In [ ]:
#recollect carc logs. Estimated time: 3min

#especific_experiment_path = "Carcassonne_full_det_1p"
#especific_experiment_path = "Carcassonne_full_sto_1p"
#find all paths in path = os.path.join("Outputs","Carcassonne_1p","Carcassonne_less_tiles_det"
paths = os.listdir(os.path.join("Outputs","Carcassonne_1p",especific_experiment_path))
number_games = 30 ######ATTENTION number of games

#remove path with DS_Store
paths = [path for path in paths if path != ".DS_Store" and ".csv" not in path and ".png" not in path]
#paths = [path for path in paths if path != "parameters.csv"]
#paths = [path for path in paths if path != "logs_by_game.csv"]
#paths = [path for path in paths if path != "logs_by_action.csv"]
#paths = [path for path in paths if path != "logs_by_iteration.csv"]
#print(paths)
for exp_path in paths:

    #Initialize final dataframes
    final_df_names = ["logs_by_game.csv", "logs_by_action.csv", "logs_by_iteration.csv"]
    final_dfs = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]

    path_outside = os.path.join("Outputs","Carcassonne_1p",especific_experiment_path,exp_path)
    print("In path: ", path_outside)
    for game_number in range(1,number_games+1):  
        #Read game data
        path = os.path.join(path_outside,"Game_"+str(game_number))
        #check if file exists
        by_dfs = []
        for df_index, df_name in enumerate(final_df_names):
            if not os.path.exists(os.path.join(path,df_name)):
                #print("File not found: ", os.path.join(path,df_name), "in path", path)
                continue
            else:
                by_dfs.append(lm.read_csv(os.path.join(path,df_name)))

        #Crate complementary data df
        complementary_data_df = lm.read_csv(os.path.join(path,"game_definition_data.csv"))
        agent_data = lm.read_csv(os.path.join(path,"p0_data.csv"))
        complementary_data_df = pd.concat([complementary_data_df, agent_data], axis=1)
        assert len(complementary_data_df) == 1
        complementary_data_df["game_number"] = [game_number]
        #add comp_ to column names
        complementary_data_df.columns = ["comp_" + str(c) for c in complementary_data_df.columns]

        #Add complementary data to each dataframe
        for df_index, df in enumerate(by_dfs):
            complementary_data_df_extended = pd.concat([complementary_data_df] * len(df), ignore_index=True)
            df = pd.concat([df, complementary_data_df_extended], axis=1)

            final_dfs[df_index] = pd.concat([final_dfs[df_index], df], ignore_index=True)

    #Add parameters data
    parameters = pd.read_csv(os.path.join(path_outside,"parameters.csv"))
    assert len(parameters) == 1
    for df in [fdf for fdf in final_dfs if len(fdf) > 0]:
        parameters_extended = pd.concat([parameters] * len(df), ignore_index=True)
        df = pd.concat([df, parameters], axis=1)
    
    #save data to dataframe
    for df_index, file_name in enumerate(final_df_names):
        if len(final_dfs[df_index]) > 0:
            lm.dump_data(data = final_dfs[df_index], file_path = path_outside, file_name = file_name)

In [ ]:
#Load and preprocess outputs. Estimated time: >15min


#experiment_path = os.path.join("Outputs","Carcassonne_1p","Carcassonne_less_tiles_det")
experiment_path = os.path.join("Outputs","Carcassonne_1p",especific_experiment_path)
logs_names = ["logs_by_game.csv", "logs_by_action.csv", "logs_by_iteration.csv"]

outputs_paths = os.listdir(experiment_path)
logs_list = {log_name:pd.DataFrame() for log_name in logs_names}
outputs_paths = [path for path in outputs_paths if path != ".DS_Store" and ".csv" not in path and ".png" not in path]

#columns to remove. # is replaced by any ints until none is found
columns_to_remove = ["tile_#_count", "action#", "action#_is_chance_node", "action#_event_type",	"action#_visits", "action#_avg_reward", "action#_tree_policy_formula", "root_node_visits", "Player", "max_fm", "max_time", "default_policy", "producing_logs", "expansion_index", "visits", "total_reward", "edge_action", "is_chance_node", "pg_player", "pg_player_name", "losing_reward", "winning_reward", "draw_reward", "no_farms", "Meeples_p2", "Score_p2", "Virtual_score_p2", "Meeples_on_farms_p2", "Total_played_meeples_p2", "playing_as"]


for path in outputs_paths:
    for log_name in logs_names:
        if not os.path.exists(os.path.join(experiment_path, path, log_name)):
            continue
        logs_list[log_name] = pd.concat([logs_list[log_name], lm.read_csv(os.path.join(experiment_path, path, log_name))], ignore_index=True)

#preprocessing
for log_name, logs in logs_list.items():    
    if len(logs) == 0:
        continue
    for column_to_int in ["pg_game_turn", "game_index", "pg_player_index"]:
        if column_to_int not in logs.columns:
            continue
        if type(logs[column_to_int].iloc[0]) is str:
            assert logs[column_to_int].iloc[0][0] == "[" and logs[column_to_int].iloc[0][-1] == "]"
            logs[column_to_int] = logs[column_to_int].apply(lambda x: int(x[1:-1]))
    temp_col = "comp_initial_meeples" #"initial_meeples"
    if temp_col in logs.columns:
        logs[temp_col] = logs[temp_col].apply(lambda x: int(x[1:-1].split(",")[0]))


for i in [1,2]: #dependency alert: names of the columns in the logs
    print("Processing meeple info from: ", logs_names[i])
    meeple_types = ["Meeple", "City", "Road", "Monastery", "Farm", "Non-Meeple"]
    info_to_track = ["_all_rewards", "_all_visits", "_average_rewards", "_std_rewards", "_average_visits", "_total_visits", "_std_visits","_max_reward", "_min_reward", "_max_visit", "_min_visit", "_alternatives_count"]
    new_data = {mt + itt : [] for mt in meeple_types for itt in info_to_track}

    for row_idx, row in logs_list[logs_names[i]].iterrows():
        names = {meeple_type:[] for meeple_type in meeple_types}

        'pg_returned_action'
        #iterate through columns
        for column_name in row.index:
            if "action" in column_name: #dependencie
                split_name = column_name.split("_")
                if len(split_name) == 1:
                    #print("found action column", column_name, row[column_name])
                    #if type(row[column_name]) is not str: print("non string argument: ", row[column_name])
                    if "Meeple Location" in str(row[column_name]): #dependency
                        names["Meeple"].append(column_name)
                        if "Farm" in row[column_name]:
                            names["Farm"].append(column_name)
                        elif "City" in row[column_name]:
                            names["City"].append(column_name)
                        elif "Road" in row[column_name]:
                            names["Road"].append(column_name)
                        elif "Monastery" in row[column_name]:
                            names["Monastery"].append(column_name)
                    else:
                        if len(str(row[column_name])) > 5:
                            names["Non-Meeple"].append(column_name)

        for meeple_type, columns in names.items():
            all_rewards = []
            all_visits = []
            for column in columns:
                #check if equals nan
                if math.isnan(row[column + "_avg_reward"]):
                    print("nan found in: ", column, "meeple_type", meeple_type, "action", row[column], row[column + "_visits"])
                all_rewards.append(row[column + "_avg_reward"])
                all_visits.append(row[column + "_visits"])
            new_data[meeple_type + "_all_rewards"].append(str(all_rewards))
            new_data[meeple_type + "_all_visits"].append(str(all_visits))
            new_data[meeple_type + "_average_rewards"].append(st.mean(all_rewards)) if len(all_rewards) > 0 else new_data[meeple_type + "_average_rewards"].append(None)
            new_data[meeple_type + "_std_rewards"].append(st.stdev(all_rewards)) if len(all_rewards) > 1 else new_data[meeple_type + "_std_rewards"].append(None)
            new_data[meeple_type + "_average_visits"].append(st.mean(all_visits)) if len(all_visits) > 0 else new_data[meeple_type + "_average_visits"].append(None)
            new_data[meeple_type + "_total_visits"].append(sum(all_visits)) if len(all_visits) > 0 else new_data[meeple_type + "_total_visits"].append(None)
            new_data[meeple_type + "_std_visits"].append(st.stdev(all_visits)) if len(all_visits) > 1 else new_data[meeple_type + "_std_visits"].append(None)
            new_data[meeple_type + "_max_reward"].append(max(all_rewards)) if len(all_rewards) > 0 else new_data[meeple_type + "_max_reward"].append(None)
            new_data[meeple_type + "_min_reward"].append(min(all_rewards)) if len(all_rewards) > 0 else new_data[meeple_type + "_min_reward"].append(None)
            new_data[meeple_type + "_max_visit"].append(max(all_visits)) if len(all_visits) > 0 else new_data[meeple_type + "_max_visit"].append(None)
            new_data[meeple_type + "_min_visit"].append(min(all_visits)) if len(all_visits) > 0 else new_data[meeple_type + "_min_visit"].append(None)
            new_data[meeple_type + "_alternatives_count"].append(len(columns))

        for meeple_type in [mt for mt in meeple_types if "Meeple" not in mt]:
            new_data[meeple_type + "_placement"] = meeple_type in row["pg_returned_action"]
        new_data["meeple_placement"] = "Meeple Location" in row["pg_returned_action"]

    new_df = pd.DataFrame(new_data)
    assert len(new_df) == len(logs_list[logs_names[i]])
    logs_list[logs_names[i]] = pd.concat([logs_list[logs_names[i]], new_df], axis=1) 

for log_name, logs in logs_list.items():

    #remove columns
    for column_to_remove in columns_to_remove:
        for column in logs.columns:
            if "#" in column_to_remove:
                for i in range(300):
                    if column_to_remove.replace("#", str(i)) == column: #in?
                        logs = logs.drop(column, axis=1)
                        #print("removed column: ", column)
            else:
                if column_to_remove == column:
                    logs = logs.drop(column, axis=1)
                    #print("removed column: ", column)
    logs_list[log_name] = logs

#save processed data
for log_name, logs in logs_list.items():
    if len(logs) == 0:
        continue
    lm.dump_data(data = logs, file_path = experiment_path, file_name = log_name)

In [ ]:
#calculate metrics

#read logs
logs_by_game = lm.read_csv(os.path.join(experiment_path,"logs_by_game.csv"))
logs_by_action = lm.read_csv(os.path.join(experiment_path,"logs_by_action.csv"))
#logs_by_iteration = lm.read_csv(os.path.join(experiment_path,"logs_by_iteration.csv"))

agent_data = {"MCTS variant":[]
              ,"Agent original name":[]
              ,"Average final score":[]
              ,"Average final score std":[]
              ,"Meeples played":[]
              ,"Meeple Efficiency Rate":[] #points scored per meeple played
              ,"Meeple Utilisation Rate":[] #meeples played / initial meeples
              ,"Meeple Availability Rate":[] #Turns with meeples
              ,"Average leaf node depth":[] #average depth of leaf nodes
              ,"City average score":[]
              ,"Road average score":[]
              ,"Farm average score":[]
              ,"Cloister average score":[]
              #,"City score percentage":[]
              ,"Average turn claiming farms":[]
              ,"Average turn claiming cities":[]
              ,"Average turn claiming roads":[]
              ,"Average turn claiming cloisters":[]
              ,"Cities claimed":[]
              ,"Farms claimed":[]
              ,"Roads claimed":[]
              ,"Cloisters claimed":[]
              ,"Cities claimed percentage":[]
              ,"Farms claimed percentage":[]
              ,"Roads claimed percentage":[]
              ,"Cloisters claimed percentage":[]

              ,"Average first turn claiming farms":[]
              ,"Average first turn claiming cities":[]
              ,"Average first turn claiming roads":[]
              ,"Average first turn claiming cloisters":[]
              ,"City percentage score":[]
              ,"Road percentage score":[]
              ,"Farm percentage score":[]
              ,"Cloister percentage score":[]
              ,"Average max reward from first level nodes":[]
              ,"Average max visits from first level nodes":[]
              ,"Clarity":[] #Average reward difference between max reward and average reward
              ,"Stubborness":[] #Average visit difference between max visits and average visits
              ,"Average available actions":[] #Average number of available actions
              ,"Initial meeples":[]
              ,"Deterministic":[]
              }
initial_meeples_list = logs_by_game["comp_initial_meeples"].unique()
is_deterministic_list = logs_by_game["comp_set_tile_sequence"].unique()
print("initial meeples list: ", initial_meeples_list, "is deterministic list: ", is_deterministic_list)
for initial_meeples in initial_meeples_list:
    f_logs_by_game = logs_by_game[logs_by_game["comp_initial_meeples"] == initial_meeples]
    print(len(f_logs_by_game), "logs by game with initial meeples: ", initial_meeples)
    f_logs_by_action = logs_by_action[logs_by_action["comp_initial_meeples"] == initial_meeples]
    print(len(f_logs_by_action), "logs by action with initial meeples: ", initial_meeples)
    for is_deterministic in is_deterministic_list:
      fd_logs_by_game = f_logs_by_game[f_logs_by_game["comp_set_tile_sequence"] == is_deterministic]
      print(len(fd_logs_by_game), "logs by game with initial meeples: ", initial_meeples, " and deterministic: ", is_deterministic)
      fd_logs_by_action = f_logs_by_action[f_logs_by_action["comp_set_tile_sequence"] == is_deterministic]
      print(len(fd_logs_by_action), "logs by action with initial meeples: ", initial_meeples, " and deterministic: ", is_deterministic)
      print("Unique agents:", fd_logs_by_game["comp_player_name"].unique())
      
      for agent_name in data_order:

          #filter data
          fda_logs_by_game = fd_logs_by_game[fd_logs_by_game["comp_player_name"] == agent_name]
          print(len(fda_logs_by_game), "logs by game with agent name: ", agent_name)
          if len(fda_logs_by_game) == 0:
             print("Agent name: " + agent_name + "Has no logs by game")
             continue
          
          agent_data["MCTS variant"].append(name_mappings[agent_name])
          agent_data["Agent original name"].append(agent_name)
          
          fda_logs_by_action = fd_logs_by_action[fd_logs_by_action["player_name"] == agent_name]
          print(len(fda_logs_by_action), "logs by action with agent name: ", agent_name)
          game_turns = sorted(fda_logs_by_action["pg_game_turn"].unique()) #statistics ignore the last two turns, asa the tree can fully explored.
          stats_game_turns = game_turns[:-2]
          stats_fda_logs_by_action = fda_logs_by_action[fda_logs_by_action["pg_game_turn"].isin(stats_game_turns)]
          print(len(stats_fda_logs_by_action), "logs by action (for stats) with initial meeples: ", initial_meeples, " and deterministic: ", is_deterministic, " and turns: ", len(stats_game_turns))
          #agent_logs_by_iteration = logs_by_iteration[logs_by_iteration["Agent"] == agent_name]
          #assert len(agent_logs_by_iteration) > 0, "Agent name: " + agent_name + "Has no logs by iteration"
          
          #calculate metrics
          avg_score = fda_logs_by_game["Score_p1"].mean()
          std_score = fda_logs_by_game["Score_p1"].std()
          played_meeples = fda_logs_by_game["Total_played_meeples_p1"].mean()
          turns_without_meeples = len(fda_logs_by_action[fda_logs_by_action["Meeples_p1"]>0])
          played_games = len(fda_logs_by_game["comp_game_number"].unique())
          turns_per_game = game_turns[-1]
          agent_max_iterations = fda_logs_by_game["comp_max_iterations"].iloc[0] if "comp_max_iterations" in fda_logs_by_game.columns else None
          #if agent_name in losing_iterations_agents:
          #    agent_max_iterations = agent_max_iterations + fda_logs_by_game["comp_es_lambda"].iloc[0] * fda_logs_by_game["comp_es_fitness_iterations"].iloc[0] * fda_logs_by_game["comp_es_generations"].iloc[0] + fda_logs_by_game["comp_es_fitness_iterations"].iloc[0]
              #print("Agent: ", agent_name, " has added max iterations: ", agent_max_iterations)
          agent_data["Average final score"].append(avg_score)
          agent_data["Average final score std"].append(std_score)
          agent_data["Meeples played"].append(played_meeples)
          agent_data["Meeple Efficiency Rate"].append(avg_score / played_meeples)
          agent_data["Meeple Utilisation Rate"].append(played_meeples / initial_meeples)
          agent_data["Meeple Availability Rate"].append((turns_without_meeples / len(game_turns))/played_games)
          agent_data["City average score"].append(fda_logs_by_game["Total_city_points_p1"].mean())
          agent_data["Road average score"].append(fda_logs_by_game["Total_road_points_p1"].mean())
          agent_data["Farm average score"].append(fda_logs_by_game["Total_Farm_points_p1"].mean())
          agent_data["Cloister average score"].append(fda_logs_by_game["Total_cloister_points_p1"].mean())
          agent_data["City percentage score"].append(fda_logs_by_game["Total_city_points_p1"].mean()/avg_score)
          agent_data["Road percentage score"].append(fda_logs_by_game["Total_road_points_p1"].mean()/avg_score)
          agent_data["Farm percentage score"].append(fda_logs_by_game["Total_Farm_points_p1"].mean()/avg_score)
          agent_data["Cloister percentage score"].append(fda_logs_by_game["Total_cloister_points_p1"].mean()/avg_score)


          agent_data["Average turn claiming farms"].append(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Farm")]["pg_game_turn"].mean()/turns_per_game)
          agent_data["Average turn claiming cities"].append(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("City")]["pg_game_turn"].mean()/turns_per_game)
          agent_data["Average turn claiming roads"].append(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Road")]["pg_game_turn"].mean()/turns_per_game)
          agent_data["Average turn claiming cloisters"].append(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Monastery")]["pg_game_turn"].mean()/turns_per_game)
          agent_data["Cities claimed"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("City")]["pg_game_turn"])/played_games)
          agent_data["Roads claimed"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Road")]["pg_game_turn"])/played_games)
          agent_data["Farms claimed"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Farm")]["pg_game_turn"])/played_games)
          agent_data["Cloisters claimed"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Monastery")]["pg_game_turn"])/played_games)
          agent_data["Cities claimed percentage"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("City")]["pg_game_turn"])/len(game_turns))
          agent_data["Roads claimed percentage"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Road")]["pg_game_turn"])/len(game_turns))
          agent_data["Farms claimed percentage"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Farm")]["pg_game_turn"])/len(game_turns))
          agent_data["Cloisters claimed percentage"].append(len(fda_logs_by_action[fda_logs_by_action["pg_returned_action"].str.contains("Monastery")]["pg_game_turn"])/len(game_turns))

          first_turn_city = []
          first_turn_road = []
          first_turn_farm = []
          first_turn_cloister = []
          for game_number in fda_logs_by_game["comp_game_number"].unique():
            filtered_run = fda_logs_by_action[fda_logs_by_action["comp_game_number"] == game_number]
            first_turn_city.append(filtered_run[filtered_run["pg_returned_action"].str.contains("City")]["pg_game_turn"].min())
            first_turn_road.append(filtered_run[filtered_run["pg_returned_action"].str.contains("Road")]["pg_game_turn"].min())
            first_turn_farm.append(filtered_run[filtered_run["pg_returned_action"].str.contains("Farm")]["pg_game_turn"].min())
            first_turn_cloister.append(filtered_run[filtered_run["pg_returned_action"].str.contains("Monastery")]["pg_game_turn"].min())
          first_turn_city = [x for x in first_turn_city if str(x) != 'nan']
          first_turn_road = [x for x in first_turn_road if str(x) != 'nan']
          first_turn_farm = [x for x in first_turn_farm if str(x) != 'nan']
          first_turn_cloister = [x for x in first_turn_cloister if str(x) != 'nan']
          agent_data["Average first turn claiming farms"].append(st.mean(first_turn_farm)/turns_per_game)
          agent_data["Average first turn claiming cities"].append(st.mean(first_turn_city)/turns_per_game)
          agent_data["Average first turn claiming roads"].append(st.mean(first_turn_road)/turns_per_game)
          agent_data["Average first turn claiming cloisters"].append(st.mean(first_turn_cloister)/turns_per_game)

          if agent_max_iterations is not None:
            agent_data["Average max reward from first level nodes"].append(stats_fda_logs_by_action["max_reward"].mean())
            agent_data["Average max visits from first level nodes"].append(stats_fda_logs_by_action["max_visits"].mean()/agent_max_iterations) #This needs to change to show propotionality with the number of actions (visits imbalance)
            agent_data["Clarity"].append(stats_fda_logs_by_action["max_reward"].mean() - stats_fda_logs_by_action["avg_reward"].mean())
            agent_data["Stubborness"].append((stats_fda_logs_by_action["max_visits"].mean() - (agent_max_iterations / stats_fda_logs_by_action["N_available_actions"].mean()))/agent_max_iterations)
            agent_data["Average leaf node depth"].append(stats_fda_logs_by_action["avg_leaf_node_depth"].mean())
          else:
            agent_data["Average max reward from first level nodes"].append(None)
            agent_data["Average max visits from first level nodes"].append(None)
            agent_data["Clarity"].append(None)
            agent_data["Stubborness"].append(None)
            agent_data["Average leaf node depth"].append(None)

          agent_data["Average available actions"].append(fda_logs_by_action["N_available_actions"].mean())
          agent_data["Initial meeples"].append(initial_meeples)
          agent_data["Deterministic"].append(is_deterministic)

          #print(agent_data)
          #raise NotImplementedError("Meeple Availability Rate")
#save metrics
metrics_df = pd.DataFrame(agent_data)
lm.dump_data(data = metrics_df, file_path = experiment_path, file_name = "metrics.csv")


In [ ]:
#statistical analysis

#load metrics
metrics_df = lm.read_csv(os.path.join(experiment_path,"logs_by_game.csv"))
agents_to_compare = ["EA_MCTS", "SIEA_MCTS"]
agent_name_column = "comp_player_name"
filter_columns = ["comp_initial_meeples", "comp_set_tile_sequence"]
comparable_column = "Score_p1"
#get all combinations of filter values
filter_values = []
for column in filter_columns:
    filter_values.append(metrics_df[column].unique())
filter_values = list(itertools.product(*filter_values))
print("filter values: ", filter_values)

comparable_lists = {a:[] for a in agents_to_compare}
#filter data
for filter_value in filter_values:
    f_metrics_df = metrics_df.copy()
    for i, column in enumerate(filter_columns):
        f_metrics_df = f_metrics_df[f_metrics_df[column] == filter_value[i]]
    print("filtered metrics df: ", len(f_metrics_df), "with filter values: ", filter_value)

    for agent_name in agents_to_compare:
        print("agent name: ", agent_name)
        a_metrics_df = f_metrics_df[f_metrics_df[agent_name_column] == agent_name]
        comparable_lists[agent_name] = list(a_metrics_df[comparable_column].values)
        
    for agent_name,value_list in comparable_lists.items():
        #check value_list is normal distributed
        stat, p = scst.normaltest(value_list)
        alpha = 0.05
        print("p = {:g}".format(p), "for agent: ", agent_name, str(p>alpha))

        #plot value_list
        fig, ax = plt.subplots(1,1)
        ax.hist(value_list, bins=20)
        ax.set_title(agent_name)
        #plt.show()

    t_stat, tp_value = scst.ttest_ind(comparable_lists[agents_to_compare[0]], comparable_lists[agents_to_compare[1]], equal_var=False)
    if tp_value < 0.05:
        print("t-test: ", agents_to_compare[0], " and ", agents_to_compare[1], " are statistically different with p-value: ", tp_value)
    else:
        print("t-test: ", agents_to_compare[0], " and ", agents_to_compare[1], " are not statistically different with p-value: ", tp_value)


In [6]:
#Observe metrics
logs = lm.read_csv(os.path.join(experiment_path,"metrics.csv"))
plot_width = 700
plot_height = 700
#xdtick = 1
ydtick = 10

#order dataset
logs["order"] = pd.Categorical(logs["Agent original name"], data_order)
logs = logs.sort_values("order")
logs = logs.dropna(subset=["order"])


column_separation_column = "Initial meeples"
row_separation_column = "Deterministic"
x_column = "MCTS variant"
bar_stack_columns = ["City average score", "Road average score", "Farm average score", "Cloister average score"]
legend_names = {"City average score":"Cities", "Road average score":"Roads", "Farm average score":"Farms", "Cloister average score":"Cloisters"}

row_identifiers = logs[row_separation_column].unique()
print("row_identifiers", row_identifiers)
column_identifiers = logs[column_separation_column].unique()
print("column_identifiers", column_identifiers)
subplot_titles = []
for row_identifier in row_identifiers:
    for column_identifier in column_identifiers:
        subplot_titles.append(carc_variants[(row_identifier, column_identifier)])
colors = eu.generate_color_palette(len(bar_stack_columns))
fig = make_subplots(rows=len(row_identifiers)
                        ,cols=len(column_identifiers)
                        ,shared_xaxes=True
                        ,shared_yaxes=True
                        ,vertical_spacing=0.04
                        ,horizontal_spacing=0.02
                        ,row_heights=[1/len(row_identifiers) for _ in range(len(row_identifiers))]
                        ,subplot_titles = subplot_titles
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Score"
                        #,y_title=y_column
                        #,print_grid=True
                        )

for row, row_identifier in enumerate(row_identifiers):
    for col, column_identifier in enumerate(column_identifiers):
        for group_id, bar_group_identifier in enumerate(bar_stack_columns):
            data = logs[logs[row_separation_column] == row_identifier]
            data = data[data[column_separation_column] == column_identifier]
            
            fig.add_trace(go.Bar(x=data[bar_group_identifier]
                                ,y=data[x_column]
                                ,name=legend_names[bar_group_identifier]
                                #, width=0.2
                                ,showlegend=(row==0 and col==0)
                                ,legendgroup=group_id
                                , marker={"color":colors[group_id],}
                                ,offsetgroup = group_id
                                ,orientation="h"
                                ,marker_pattern_shape=pattern_shapes[group_id]
                                )
                            ,row=row+1
                            ,col=col+1)

fig.update_layout(margin=dict(l=70, r=30, t=30, b=70)
                        ,width=plot_width
                        ,height=plot_height
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        #,boxmode='group'
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,boxgap=0.3
                        ,boxgroupgap=0
                        ,legend=dict(
                            title = "MCTS variant"
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.1
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            ) 
                        )
fig.update_layout(barmode='stack')
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        #,range=[0, 10]
                        ,showgrid=True
                        ,gridcolor="grey"
                        ,gridwidth=1
                        #,dtick= xdtick
                        #,tick0=0
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    #,tick0 = 0
                    , gridcolor="#5B8C5A"
                    , gridwidth=1
                    #, dtick=ydtick
                    ,showgrid=False                    
                    )
#fig.write_image(os.path.join(experiment_path, "c1p_score_distribution.png"))
fig.show()

row_identifiers [ True False]
column_identifiers [1 3]


In [ ]:
#Plot for action level logs by turn

setups = None# [0,3]
#setups = [6,7]
#defaults:
plot_width = 800
plot_height = 800
ignore_last_turns = None

if setups is None:
    setups = range(8)
for setup in setups:
    if setup == 0:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "avg_leaf_node_depth"
        shadows = False
        ydtick=3
        x_range = [0,24]
        y_range = [0,11]

    if setup == 1:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"#"current_iterations"#"pg_game_turn"
        y_column = "Score_p1"
        shadows = False
        ydtick=3

    if setup == 2:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "Meeples_on_farms_p1"
        shadows = False
        ydtick = 1

    if setup == 3:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "max_visits"
        shadows = False
        ydtick = 1000

    if setup == 4:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "Total_played_meeples_p1"
        shadows = False
        ydtick = 1

    if setup == 5:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "Virtual_score_p1"
        shadows = False
        ydtick=3

    if setup == 6:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "Meeples_p1"
        shadows = False
        ydtick=1
    if setup == 7:
        #define variables
        to_show = "Average"
        box_group_column = "player_name"
        x_column = "pg_game_turn"
        y_column = "forward_model_calls"
        shadows = False
        ydtick=100000

    #try:
    column_separation_column = "comp_initial_meeples"
    row_separation_column = "comp_set_tile_sequence"
    xdtick = 2

    #read data
    logs_names = ["logs_by_action.csv"]#, "logs_by_action.csv", "logs_by_iteration.csv"]
    logs_list = {logs_name:lm.read_csv(os.path.join(experiment_path, logs_name)) for logs_name in logs_names}
    logs_index = 0
    logs = logs_list[logs_names[logs_index]]
    #filter data
    logs = logs[logs["comp_initial_meeples"]!=2]
    if ignore_last_turns is not None:
        logs = logs[~logs["pg_game_turn"].isin(range(max(logs["pg_game_turn"]) - ignore_last_turns, max(logs["pg_game_turn"])+1))]

    #order dataset
    logs["order"] = pd.Categorical(logs["player_name"], data_order)
    logs = logs.sort_values("order")
    logs = logs.dropna(subset=["order"])

    #Calculate uniques
    row_identifiers = sorted(logs[row_separation_column].unique(), reverse=True)
    column_identifiers = sorted(logs[column_separation_column].unique())
    box_group_identifiers = logs[box_group_column].unique()
    colors = eu.generate_color_palette(len(box_group_identifiers))
    #colors = ["#5B8C5A","#56638A", "#EC7316"]
    print("row_identifiers", row_identifiers)
    print("column_identifiers", column_identifiers)
    print("box_group_identifiers", box_group_identifiers)

    subplot_titles = []
    for row_identifier in row_identifiers:
        for column_identifier in column_identifiers:
            subplot_titles.append(carc_variants[(row_identifier, column_identifier)])#+str(row_identifier) + "m" + str(column_identifier) + "ts,")

    #Create figure
    fig = make_subplots(rows=len(row_identifiers)
                            ,cols=len(column_identifiers)
                            ,shared_xaxes=True
                            ,shared_yaxes=True
                            ,vertical_spacing=0.04
                            ,horizontal_spacing=0.02
                            ,row_heights=[1/len(row_identifiers) for _ in range(len(row_identifiers))]
                            ,subplot_titles = subplot_titles
                            #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                            ,x_title=name_mappings[x_column]
                            ,y_title=name_mappings[y_column]
                            #,print_grid=True
                            )

    max_x_list = []
    max_y_list = []

    for row, row_identifier in enumerate(row_identifiers):
        for col, column_identifier in enumerate(column_identifiers):
            for group_id, box_group_identifier in enumerate(box_group_identifiers):
                data = logs[logs[row_separation_column] == row_identifier]
                data = data[data[column_separation_column] == column_identifier]
                data = data[data[box_group_column] == box_group_identifier]
                if len(data) == 0:
                    continue
                if to_show == "Average":
                    x = list(data[x_column].unique())
                    max_x_list.append(max(x))
                    #sort x:
                    x = sorted(x)
                    y1 = []
                    y1_lower = []
                    y1_upper = []
                    for x_unique in x:
                        temp_data = data[data[x_column] == x_unique]
                        if len(temp_data) == 0:
                            continue
                        values = temp_data[y_column]
                        #remove NaN from values
                        values = [v for v in values if not math.isnan(v)]
                        if len(values) == 0:
                            continue
                        current_mean = st.mean(values)
                        y1.append(current_mean)
                        std = st.stdev(values)
                        #y_lower.append()
                        y1_lower.append(current_mean - std / math.sqrt(len(values)))
                        y1_upper.append(current_mean + std / math.sqrt(len(values)))
                        #print(y1_lower[-1], y1[-1], y1_upper[-1])
                    if len(y1) > 0:
                        if shadows:
                        #Prepare data for plot
                            x_rev = x[::-1]
                            y1_lower = y1_lower[::-1]

                            #Plot shadow
                            shadow_color = eu.color_rgb_to_hex(eu.brighten_color(eu.color_hex_to_rgb(colors[group_id]), 0.3))
                            fig.add_trace(go.Scatter(
                                    x=x+x_rev
                                    ,y=y1_upper+y1_lower
                                    ,fill='toself'
                                    ,fillcolor=colors[group_id]
                                    ,line_color=colors[group_id]
                                    ,showlegend=False
                                    ,opacity = 0.3
                                    #,marker={"color":colors[group_id]}
                                    )
                                ,row=row+1
                                ,col=col+1)
                    
                        #LINE
                        fig.add_trace(go.Scatter(
                                x=x
                                ,y=y1
                                #,line_color=colors[group_id]
                                #,line=dict(dash="solid")
                                ,mode='lines+markers'
                                ,name=name_mappings[box_group_identifier]
                                ,showlegend=(row==0 and col==0)
                                ,legendgroup=group_id
                                ,marker=dict(symbol=marker_symbols[group_id], size=7, color =colors[group_id])
                                )
                            ,row=row+1
                            ,col=col+1)
                        max_y_list.append(max(y1))
                elif to_show == "Boxplot":
                    fig.add_trace(go.Box(x=data[x_column]
                                        ,y=data[y_column]
                                        ,name=name_mappings[box_group_identifier]
                                        #, width=0.2
                                        ,showlegend=(row==0 and col==0)
                                        ,legendgroup=group_id
                                        , marker={"color":colors[group_id],}
                                        ,offsetgroup = group_id
                                        )
                                    ,row=row+1
                                    ,col=col+1)

    x_range = [0, max(max_x_list)]
    y_range = [0, max(max_y_list)+0.0001]

    fig.update_layout(margin=dict(l=70, r=30, t=30, b=70)
                            ,width=plot_width
                            ,height=plot_height
                            ,plot_bgcolor='rgba(0,0,0,0)'
                            #,plot_bgcolor="lightgray"
                            #,boxmode='group'
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,boxgap=0.3
                            ,boxgroupgap=0
                            ,legend=dict(
                                title = "Agent"
                                ,orientation="h"
                                ,yanchor="top"
                                ,y=-0.1
                                #,xanchor="center"
                                #,x=0.4
                                ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                                ,font = dict(family = "Arial", size = 14, color = "black")
                                ,bordercolor="Black"
                                ,borderwidth=2
                                ,itemsizing='trace'
                                ,itemwidth = 30
                                ) 
                            )
    if to_show == "Boxplot":
        fig.update_layout(boxmode='group')
    fig.update_xaxes(showline=True
                            , linewidth=2
                            , linecolor='black'
                            , mirror=True
                            #,showgrid=True
                            #,gridcolor="grey"
                            #,gridwidth=1
                            ,dtick= xdtick
                            ,tick0=0
                            ,range=x_range
                            )

    fig.update_yaxes(showline=True
                       , mirror=True
                        , linewidth=2
                        , linecolor='black'
                        #, nticks=5
                        #,tickmode = 'linear'
                        ,tick0 = 0
                        , gridcolor="grey"#"#5B8C5A"
                        , gridwidth=0.5
                        #, dtick= 3 if ydtick is None else ydtick
                        #,showgrid=True  
                        ,range=y_range                
                                          
                        )
    fig.write_image(os.path.join(experiment_path, "c1p_" + name_mappings[y_column].replace(" ","_") +".png"))
    #fig.show()
    #except Exception as e:
    #    print("Error in setup: ", setup)
    #    print(e)
    #    continue

In [44]:
#time estimator

games = 30
variants = 5
turns = 23
time_limit = 15
cores = 6
runs = 30

total_time = ((games * runs*  variants * turns * time_limit / cores) / 60) / 60
print("Total time: ", total_time, "hours")

Total time:  71.875 hours


In [70]:
from PIL import Image
#from IPython.display import Image# as DImage

def convert_to_grayscale(image_path):
    # Open the image file
    with DImage.open(image_path) as img:
        # Convert the image to grayscale
        grayscale_img = img.convert("L")

        # Save the grayscale image
        grayscale_img_path = image_path.replace('.png', '_grayscale.png')
        grayscale_img.save(grayscale_img_path)

        return grayscale_img_path

def convert_png_to_eps(png_file, eps_file):
    # Open the PNG image
    image = Image.open(png_file)

    # Convert the image to RGB mode if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Save the image as EPS
    image.save(eps_file, format='eps')

# Example usage
im_names = ["c1p_Meeples_on_farms", "c1p_Total_meeples_played","c1p_score_distribution","c1p_Average_leaf_node_depth","c1p_Most_visited_node_visit_count"]
for im_name in im_names:
    im_path = os.path.join(experiment_path, im_name + ".png")
    out_path = os.path.join(experiment_path, im_name + ".eps")
    convert_png_to_eps(im_path, out_path)

#converted_image_path = convert_to_grayscale(os.path.join(experiment_path, "c1p_" + "score_distribution" +".png"))
#view image
#from IPython.display import Image
#Image(filename=converted_image_path)


In [17]:
#Plot for game level logs

plot_width = 800
plot_height = 500

for setup in [0,1]:
    if setup == 0:
        #define variables
        box_group_column = None
        x_column = "Player_0"
        y_column = "Score_p1"
        shadows = False
        xdtick=10
    if setup == 1:
        #define variables
        box_group_column = None
        x_column = "Player_0"
        y_column = "Total_played_meeples_p1"
        shadows = False
        xdtick=1

    to_show = "Boxplot"
    column_separation_column = "comp_initial_meeples"
    row_separation_column = "comp_set_tile_sequence"

    #read data
    logs_names = ["logs_by_game.csv"]#, "logs_by_action.csv", "logs_by_iteration.csv"]
    logs_list = {logs_name:lm.read_csv(os.path.join(experiment_path, logs_name)) for logs_name in logs_names}
    logs_index = 0
    logs = logs_list[logs_names[logs_index]]
    #filter data
    logs = logs[logs["comp_initial_meeples"]!=2]

    #order dataset
    logs["order"] = pd.Categorical(logs["Player_0"], data_order)
    logs = logs.sort_values("order")
    logs = logs.dropna(subset=["order"])

    #change names
    if x_column == "Player_0":
        logs[x_column] = logs[x_column].apply(lambda x: name_mappings[x])

    #Calculate uniques
    row_identifiers = sorted(logs[row_separation_column].unique(), reverse=True)
    column_identifiers = sorted(logs[column_separation_column].unique())
    box_group_identifiers = logs[box_group_column].unique() if box_group_column is not None else None
    colors = eu.generate_color_palette(1)
    #colors = ["#5B8C5A","#56638A", "#EC7316"]
    print("row_identifiers", row_identifiers)
    print("column_identifiers", column_identifiers)
    print("box_group_identifiers", box_group_identifiers)

    subplot_titles = []
    for row_identifier in row_identifiers:
        for column_identifier in column_identifiers:
            subplot_titles.append(carc_variants[(row_identifier, column_identifier)])#+str(row_identifier) + "m" + str(column_identifier) + "ts,")

    #Create figure
    fig = make_subplots(rows=len(row_identifiers)
                            ,cols=len(column_identifiers)
                            ,shared_xaxes=True
                            ,shared_yaxes=True
                            ,vertical_spacing=0.07
                            ,horizontal_spacing=0.02
                            ,row_heights=[1/len(row_identifiers) for _ in range(len(row_identifiers))]
                            ,subplot_titles = subplot_titles
                            #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                            ,x_title=name_mappings[y_column]
                            #,y_title=name_mappings[y_column]
                            #,print_grid=True
                            )

    for row, row_identifier in enumerate(row_identifiers):
        for col, column_identifier in enumerate(column_identifiers):
            #for group_id, box_group_identifier in enumerate(box_group_identifiers):
                data = logs[logs[row_separation_column] == row_identifier]
                data = data[data[column_separation_column] == column_identifier]
                #data = data[data[box_group_column] == box_group_identifier]
                
                fig.add_trace(go.Box(x=data[y_column]
                                    ,y=data[x_column]
                                    #,name=name_mappings[box_group_identifier]
                                    #, width=0.2
                                    ,showlegend=False#(row==0 and col==0)
                                    #,legendgroup=group_id
                                    , marker={"color":colors[0],}
                                    ,offsetgroup = 1#group_id
                                    ,orientation="h"
                                    )
                                ,row=row+1
                                ,col=col+1)

    fig.update_layout(margin=dict(l=70, r=30, t=30, b=70)
                            ,width=plot_width
                            ,height=plot_height
                            ,plot_bgcolor='rgba(0,0,0,0)'
                            #,plot_bgcolor="lightgray"
                            #,boxmode='group'
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,boxgap=0.3
                            ,boxgroupgap=0
                            ,legend=dict(
                                title = "MCTS variant"
                                ,orientation="h"
                                ,yanchor="top"
                                ,y=-0.1
                                #,xanchor="center"
                                #,x=0.4
                                ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                                ,font = dict(family = "Arial", size = 14, color = "black")
                                ,bordercolor="Black"
                                ,borderwidth=2
                                ,itemsizing='trace'
                                ,itemwidth = 30
                                ) 
                            )
    if to_show == "Boxplot":
        fig.update_layout(boxmode='group')
    fig.update_xaxes(showline=True
                            , linewidth=2
                            , linecolor='black'
                            , mirror=True
                            #,range=[0, 10]
                            ,showgrid=True
                            ,gridcolor="grey"
                            ,gridwidth=1
                            ,dtick= xdtick
                            ,tick0=0
                            )

    fig.update_yaxes(showline=True
                        ,mirror=True
                        , linewidth=2
                        , linecolor='black'
                        #, nticks=5
                        #,tickmode = 'linear'
                        ,tick0 = 0
                        , gridcolor="#5B8C5A"
                        , gridwidth=1
                        #, dtick= 3 if ydtick is None else ydtick
                        ,showgrid=True                    
                        )
    fig.write_image(os.path.join(experiment_path, "c1p_bygame_" + name_mappings[y_column] +".png"))
    fig.show()

row_identifiers [True, False]
column_identifiers [1, 3]
box_group_identifiers None


row_identifiers [True, False]
column_identifiers [1, 3]
box_group_identifiers None


In [11]:
#plots game level by game

#define variables
to_show = "Boxplot" #Average or Boxplot
#plot_separation_column = ["comp_initial_meeples", "comp_set_tile_sequence"]
row_separation_column = "Player_0"
column_separation_column = None#"comp_set_tile_sequence"
box_group_column = None#"Player_0"#"player_name"
x_column = "comp_game_number"#"current_iterations"#"pg_game_turn"
y_column = "Score_p1" #avg_leaf_node_depth
#y_column = "avg_leaf_node_depth"

#read data
logs_names = ["logs_by_game.csv"]#, "logs_by_action.csv", "logs_by_iteration.csv"]
logs_list = {logs_name:lm.read_csv(os.path.join(experiment_path, logs_name)) for logs_name in logs_names}
logs_index = 0
logs = logs_list[logs_names[logs_index]]
#filter data
logs = logs[logs["comp_initial_meeples"]==3]
logs = logs[logs["comp_set_tile_sequence"]==True]
print(logs["comp_set_tile_sequence"].unique())
print("logs length after filtering: ", len(logs))

#if len(plot_separation_column) > 1:
#    logs["plot_separation_column"] = logs[plot_separation_column[0]].astype(str) + " " + logs[plot_separation_column[1]].astype(str)
#    plot_separation_column = "plot_separation_column"

#order dataset
logs["order"] = pd.Categorical(logs["Player_0"], data_order)
logs = logs.sort_values("order")
logs = logs.dropna(subset=["order"])

#print(logs_list[logs_names[logs_index]].head())
#Calculate uniques
#plot_identifiers = logs[plot_separation_column].unique()
row_identifiers = logs[row_separation_column].unique() if row_separation_column is not None else [None]
column_identifiers = logs[column_separation_column].unique() if column_separation_column is not None else [None]
box_group_identifiers = logs[box_group_column].unique() if box_group_column is not None else [None]
colors = eu.generate_color_palette(len(box_group_identifiers)) if box_group_column is not None else ["#5B8C5A"]
#colors = ["#5B8C5A","#56638A", "#EC7316"]
print("row_identifiers", row_identifiers)
print("column_identifiers", column_identifiers)
print("box_group_identifiers", box_group_identifiers)

#Create figure
fig = make_subplots(rows=len(row_identifiers)
                        ,cols=len(column_identifiers)
                        ,shared_xaxes=True
                        ,shared_yaxes=True
                        ,vertical_spacing=0.04
                        ,row_heights=[1/len(row_identifiers) for _ in range(len(row_identifiers))]
                        ,subplot_titles = [str(row_separation_column) + str(r) + str(column_separation_column) + str(c) for r in row_identifiers for c in column_identifiers]
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Score"
                        #,y_title='Score'
                        #,print_grid=True
                        )

for row, row_identifier in enumerate(row_identifiers):
    for col, column_identifier in enumerate(column_identifiers):
    #for group_id, box_group_identifier in enumerate(box_group_identifiers):
        data = logs[logs[row_separation_column] == row_identifier] if row_separation_column is not None else logs
        data = data[data[column_separation_column] == column_identifier] if column_separation_column is not None else data
        
        fig.add_trace(go.Bar(x=data[x_column]
                            ,y=data[y_column]
                            ,orientation='v'
                            #,name=str(box_group_identifier)
                            , width=0.8
                            ,showlegend=False
                            #,legendgroup=str(box_group_identifier)
                            , marker={"color":colors[0]}
                            )
                        ,row=row+1
                        ,col=col+1)

fig.update_layout(margin=dict(l=70, r=10, t=30, b=50)
                        ,width=1200
                        ,height=700
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        #,boxmode='group'
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,legend=dict(
                            title = "Scores by MCTS variant"
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.075
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            ) 
                        )
if to_show == "Average":
    fig.update_traces(mode='lines')
elif to_show == "Boxplot":
    fig.update_layout(boxmode='group')
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    #,tick0 = 0
                    , gridcolor="#5B8C5A"
                    , gridwidth=0.1
                    #, dtick=5000
                    ,showgrid=True
                    )
fig.show()

[ True]
logs length after filtering:  160
logs length after ordering:  160
row_identifiers ['EA_MCTS2_its7430', 'EA_MCTS_its5000', 'MCTS_c3', 'MCTS_c0.5', NaN]
Categories (5, object): ['EA_MCTS2_its7430', 'EA_MCTS_its5000', 'MCTS_c3', 'MCTS_c0.5', 'MCTS_c0_5']
column_identifiers [None]
box_group_identifiers [None]


In [27]:
#plots game level by game

#define variables
row_separation_column = "comp_initial_meeples"
column_separation_column = "comp_set_tile_sequence"
box_group_column = "Player_0"#"Player_0"#"player_name"
x = ["Farm","city","road","cloister"]
x_column = "Feature"
y_column = "Total_score"

#read data
logs = lm.read_csv(os.path.join(experiment_path, "logs_by_game.csv"))
#filter data
#logs = logs[logs["comp_initial_meeples"]==3]
#logs = logs[logs["comp_set_tile_sequence"]==True]
#print(logs["comp_set_tile_sequence"].unique())
#print("logs length after filtering: ", len(logs))
#create data
new_data_dict = {"Total_score":[],"Incomplete_score":[],"Completed_score":[],"Feature":[],"Player_0":[],"comp_initial_meeples":[], "comp_set_tile_sequence":[], "Score_p1":[], "Iterations":[], "comp_game_number":[]}
for x_name in x:
    for row in logs.iterrows():
        new_data_dict["Feature"].append(x_name)
        new_data_dict["Player_0"].append(row[1]["Player_0"])
        new_data_dict["Total_score"].append(row[1]["Total_" + x_name + "_points_p1"])
        new_data_dict["comp_initial_meeples"].append(row[1]["comp_initial_meeples"])
        new_data_dict["comp_set_tile_sequence"].append(row[1]["comp_set_tile_sequence"])
        new_data_dict["Score_p1"].append(row[1]["Score_p1"])
        new_data_dict["Iterations"].append(row[1]["comp_max_iterations"])
        new_data_dict["comp_game_number"].append(row[1]["comp_game_number"])
        if x_name != "Farm":
            new_data_dict["Incomplete_score"].append(row[1]["Uncompleted_" + x_name + "_points_p1"])
            new_data_dict["Completed_score"].append(row[1]["Completed_" + x_name + "_points_p1"])
        else:
            new_data_dict["Incomplete_score"].append(0)
            new_data_dict["Completed_score"].append(0)
logs = pd.DataFrame(new_data_dict)
print("logs length after creating data: ", len(logs))

#order dataset
logs["order"] = pd.Categorical(logs["Player_0"], data_order)
logs = logs.sort_values("order")
logs = logs.dropna(subset=["order"])
order_identifiers = logs["order"].unique()
print("order identifiers: ", order_identifiers)

#save dataset
lm.dump_data(data = logs, file_path = experiment_path, file_name = "logs_by_feature_and_game.csv")

#print(logs_list[logs_names[logs_index]].head())
#Calculate uniques
row_identifiers = logs[row_separation_column].unique() if row_separation_column is not None else [None]
column_identifiers = logs[column_separation_column].unique() if column_separation_column is not None else [None]
box_group_identifiers = logs[box_group_column].unique() if box_group_column is not None else [None]
colors = eu.generate_color_palette(len(box_group_identifiers)) if box_group_column is not None else ["#5B8C5A"]
#colors = ["#5B8C5A","#56638A", "#EC7316"]
print("row_identifiers", row_identifiers)
print("column_identifiers", column_identifiers)
print("box_group_identifiers", box_group_identifiers)

#Create figure
fig = make_subplots(rows=len(row_identifiers)
                        ,cols=len(column_identifiers)
                        ,shared_xaxes=True
                        ,shared_yaxes=True
                        ,vertical_spacing=0.04
                        ,row_heights=[1/len(row_identifiers) for _ in range(len(row_identifiers))]
                        ,subplot_titles = [str(row_separation_column) + str(r) + str(column_separation_column) + str(c) for r in row_identifiers for c in column_identifiers]
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Score"
                        #,y_title='Score'
                        #,print_grid=True
                        )

for row, row_identifier in enumerate(row_identifiers):
    for col, column_identifier in enumerate(column_identifiers):
        for group_id, box_group_identifier in enumerate(box_group_identifiers):
            data = logs[logs[row_separation_column] == row_identifier] if row_separation_column is not None else logs
            data = data[data[column_separation_column] == column_identifier] if column_separation_column is not None else data
            data = data[data[box_group_column] == box_group_identifier] if box_group_column is not None else data
            print("Data length: ", len(data), "in" , row_identifier, column_identifier, box_group_identifier, "Score avg", str(st.mean(data[y_column])))
            fig.add_trace(go.Box(x=data[x_column]
                                ,y=data[y_column]
                                ,orientation='v'
                                ,name=str(box_group_identifier)
                                #, width=0.8
                                ,showlegend=(row==0 and col==0)
                                ,legendgroup=str(box_group_identifier)
                                , marker={"color":colors[group_id]}
                                ,offsetgroup=group_id
                                )
                            ,row=row+1
                            ,col=col+1)

fig.update_layout(margin=dict(l=70, r=10, t=30, b=50)
                        ,width=1000
                        ,height=600
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        #,boxmode='group'
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,legend=dict(
                            title = "Scores by MCTS variant"
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.075
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            ) 
                        )
fig.update_layout(boxmode='group')
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        )

fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    #, nticks=5
                    #,tickmode = 'linear'
                    #,tick0 = 0
                    , gridcolor="#5B8C5A"
                    , gridwidth=0.1
                    #, dtick=5000
                    ,showgrid=True
                    )
fig.show()


logs length after creating data:  2560
order identifiers:  ['EA_MCTS2_its7430', 'EA_MCTS_its5000', 'MCTS_c3', 'MCTS_c0.5']
Categories (4, object): ['EA_MCTS2_its7430', 'EA_MCTS_its5000', 'MCTS_c3', 'MCTS_c0.5']
row_identifiers [3 1]
column_identifiers [False  True]
box_group_identifiers ['EA_MCTS2_its7430' 'EA_MCTS_its5000' 'MCTS_c3' 'MCTS_c0.5']
Data length:  80 in 3 False EA_MCTS2_its7430 Score avg 11.8375
Data length:  80 in 3 False EA_MCTS_its5000 Score avg 11.8
Data length:  80 in 3 False MCTS_c3 Score avg 12.1875
Data length:  80 in 3 False MCTS_c0.5 Score avg 12.1125
Data length:  80 in 3 True EA_MCTS2_its7430 Score avg 11.9625
Data length:  80 in 3 True EA_MCTS_its5000 Score avg 12.4375
Data length:  80 in 3 True MCTS_c3 Score avg 11.75
Data length:  80 in 3 True MCTS_c0.5 Score avg 12.15
Data length:  80 in 1 False EA_MCTS2_its7430 Score avg 7.725
Data length:  80 in 1 False EA_MCTS_its5000 Score avg 6.4
Data length:  80 in 1 False MCTS_c3 Score avg 6.825
Data length:  80 in 1

In [ ]:
#recollect carc logs

#find all paths in path = os.path.join("Outputs","Carcassonne_1p","Carcassonne_less_tiles_det"
paths = os.listdir(os.path.join("Outputs","Carcassonne_1p","Carcassonne_less_tiles_det"))
#print(paths)
for exp_path in paths:

    #Initialize final dataframes
    final_df_names = ["logs_by_game.csv", "logs_by_action.csv", "logs_by_iteration.csv"]
    final_dfs = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]

    path_outside = os.path.join("Outputs","Carcassonne_1p","Carcassonne_less_tiles_det",exp_path)
    print("In path: ", path_outside)
    for game_number in range(1,21):

        #Read game data
        path = os.path.join(path_outside,"Game_"+str(game_number))
        by_dfs = [lm.read_csv(os.path.join(path,logs_name)) for logs_name in final_df_names]

        #Crate complementary data df
        complementary_data_df = lm.read_csv(os.path.join(path,"game_definition_data.csv"))
        assert len(complementary_data_df) == 1
        complementary_data_df["game_number"] = [game_number]

        #Add complementary data to each dataframe
        for df_index, df in enumerate(by_dfs):
            complementary_data_df_extended = pd.concat([complementary_data_df] * len(df), ignore_index=True)
            df = pd.concat([df, complementary_data_df_extended], axis=1)
            #check for duplicated colums
            #print("Printing duplicated columns within each df:")
            #print(df.columns[df.columns.duplicated()])
            #print(final_dfs[df_index].columns[final_dfs[df_index].columns.duplicated()])
            #print("Overlapping:", df.columns.intersection(final_dfs[df_index].columns))

            final_dfs[df_index] = pd.concat([final_dfs[df_index], df], ignore_index=True)

    #Add parameters data
    parameters = pd.read_csv(os.path.join(path_outside,"parameters.csv"))
    assert len(parameters) == 1
    for df in final_dfs:
        parameters_extended = pd.concat([parameters] * len(df), ignore_index=True)
        df = pd.concat([df, parameters], axis=1)
    
    #save data to dataframe
    for df_index, file_name in enumerate(final_df_names):
        lm.dump_data(data = final_dfs[df_index], file_path = path_outside, file_name = file_name)

    #break
#print(df_by_game["Score_p1"])
#print mean score
#print(pd.read_csv(os.path.join("Outputs","Carcassonne_1p","Carcassonne_less_tiles_det",exp_path,"parameters.csv")))
#print(df_by_game["Score_p1"].mean())
#print(df_by_game["game_time"].mean())
#print(df_by_game["Total_played_meeples_p1"].mean())

In [2]:
#game_state = mnk.GameState(3,3,3)
game_state = carc.CarcassonneState(set_tile_sequence=True, set_tile_sequence_seed=0, players = 1, initial_meeples=[1,1])
game_state.set_initial_state()
agent = mcts.MCTS_Player(max_iterations = 50, logs = True, logs_every_iterations=10)
gp = eu.GamePlayer(game_state, [agent])
gp.play_game(random_seed=0)
gp.save_data(file_path = os.path.join("Outputs", "test", "gp_iteration_logs"))
#action = agent.choose_action(game_state)
#agent.dump_my_logs(path = os.path.join("Outputs", "test", "new_iteration_logs"))

In [ ]:
#One player test
game_state = carc.CarcassonneState(name = "Carcassonne_1p_set", set_tile_sequence=True, set_tile_sequence_seed=0, initial_meeples = [1], players = 1)
game_state.set_initial_state()
#players = [arand.RandomPlayer()]
players = [mcts.MCTS_Player(max_iterations =50)]
gameplayer = eu.GamePlayer(game_state, players)
gameplayer.play_game()
gameplayer.logs_by_game
#get detailed logs on first decision

In [6]:
gameplayer.save_data(file_path = os.path.join("Outputs","test","Carcassonne_1p_vanilla"))

In [ ]:
#Small carcassonne
games = 10
date_time_string = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
game_state = carc.CarcassonneState(set_tile_sequence=True, set_tile_sequence_seed=1, initial_meeples=[1,1], initial_tile_quantities=[1 for _ in range(24)])
game_state.set_initial_state()
agents = [mcts.MCTS_Player(max_iterations=5000, c=0.5, name="MCTS_c0.5", logs = True), siea_mcts.SIEA_MCTS_Player(max_iterations=5000, name="SIEA_MCTS_its2570", logs=True, use_semantics=True)]
gp1, gp2 = eu.play_match(agents, game_state, games, os.path.join("Outputs", "Matches", "Carcassonne_fixed_sequence_" + date_time_string), random_seed = 0)
#gp1.save_data(file_path = os.path.join("Outputs", "Matches", "Carcassonne_fixed_sequence_" + date_time_string))
#cProfile.run("gp1, gp2 = eu.play_match(agents, game_state, 2, os.path.join('Outputs', 'Matches', 'Carcassonne_fixed_sequence_' + date_time_string), random_seed = 2)")

In [ ]:
#Create color palette

color_palette = [ "#B10909" #red
                ,  "#5B8C5A"#green
                ,"#56638A" #blue-purple
                , "#EC7316" #orange
                ,  "#FC738C" ] #pink
color_palette = ["#5B8C5A"
                ,"#56638A"
                , "#EC7316"]
color_palette[2] = eu.color_rgb_to_hex(eu.darken_color(eu.color_hex_to_rgb(color_palette[2]), 0.4))
color_palette[0] = eu.color_rgb_to_hex(eu.brighten_color(eu.color_hex_to_rgb(color_palette[0]), 0.3))
color_palette[1] = eu.color_rgb_to_hex(eu.brighten_color(eu.color_hex_to_rgb(color_palette[1]), 0.1))

eu.view_color_palette(color_palette)

print("color_palette = {}".format(color_palette))

-----------FOP-------------

In [2]:
#General parameters
#logs_path = os.path.join("Outputs","FO_single_decision_final")
logs_path = os.path.join("Outputs","FOP")
data_order = [
        "MCTS_c0_5",
        "MCTS_c1",
        "MCTS_c1_4142",
        "MCTS_c2",
        "MCTS_c3",
        #"EA_MCTS_its2570",
        "EA_MCTS_its5000",
        #"SIEA_MCTS_its2570",
        "SIEA_MCTS_its5000",
        #"SIEA2_its2570",
        #"SIEA2_its5000",
        #"SIEA2_MCTS_its2570",
        #"SIEA2_MCTS_its5000",
        #"SIEA2_MCTS_its7430",
        #"SIEA2_MCTS_discard",
        "REA_MCTS__PE_False__NP_False__RE_False",
        "REA_MCTS__PE_False__NP_False__RE_True",
        "REA_MCTS__PE_False__NP_True__RE_False",
        "REA_MCTS__PE_False__NP_True__RE_True",
        "REA_MCTS__PE_True__NP_False__RE_False",
        "REA_MCTS__PE_True__NP_False__RE_True",
        "REA_MCTS__PE_True__NP_True__RE_False",
        "REA_MCTS__PE_True__NP_True__RE_True",
    ]
data_order = data_order[::-1]
colors = ["#5B8C5A","#56638A","#EC7316","#e4c9db","#e4deb8","#e265ef","#c27dd1","#d52e87","#a8d0b9","#ad8dfd"]
name_replaces = {"SIEA2_MCTS_its2570": "EA_MCTS2_its2570", "SIEA2_MCTS_its5000": "EA_MCTS2_its5000", "SIEA2_MCTS_its7430": "EA_MCTS2_its7430", "EA_MCTS_its2570": "EA_MCTS", "EA_MCTS_its5000": "EA_MCTS", "SIEA_MCTS_its5000": "SIEA_MCTS", "SIEA_MCTS_its2570": "SIEA_MCTS", "SIEA2_MCTS_discard":"EA_MCTS2_justdiscard"}

In [32]:
#Compile logs - needed only the first time! - expected run time: <1min

for file_name in ["evolution_logs.csv", "results.csv", "logs_by_run.csv"]:
    file_path_list = lm.find_log_files(file_name, logs_path)
    lm.combine_logs(logs_path, file_name, file_path_list)

#Evolved formulas analysis
data = pd.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
evolved_formula_data = pd.DataFrame()
for agent in data["Player"].unique():
    if "EA" in agent:
        for f_index in data["Function_index"].unique():
            tdata = data[(data["Player"]==agent) & (data["Function_index"]==f_index)]
            fa_data = eu.evolved_formula_analysis(tdata)
            fa_data["Player"] = [agent]
            fa_data["Function_index"] = [f_index]
            evolved_formula_data = pd.concat([evolved_formula_data, fa_data])
evolved_formula_data.to_csv(os.path.join(logs_path, "evolved_formula_analysis.csv"))

In [4]:
#Collapsed logs by run -- Needed to get the final results
data = pd.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
avg_columns = ["forward_model_calls","root_node_avg_reward","nodes_count","avg_leaf_node_depth","max_leaf_node_depth","min_leaf_node_depth", "std_leaf_node_depth", "leaf_node_count","max_visits","max_reward","evolution_fm_calls","avg_total_nodes"]
tdata_dict = {col:[] for col in avg_columns}
tdata_dict2 = {col+"_collapsed_std":[] for col in avg_columns}
tdata_dict = {**tdata_dict, **tdata_dict2}
tdata_dict["Player"] = []
tdata_dict["Function_index"] = []
for agent in data["Player"].unique():
    for function_index in data["Function_index"].unique():
        tdata = data[(data["Player"]==agent) & (data["Function_index"]==function_index)]
        #print(len(tdata))
        tdata_dict["Player"] += [agent]
        tdata_dict["Function_index"] += [function_index]
        for column in avg_columns:
            tdata_dict[column] += [tdata[column].mean()]
            tdata_dict[column + "_collapsed_std"] += [tdata[column].std()]
        #print(len(tdata_dict["Player"]))
tdata = pd.DataFrame(tdata_dict)
lm.dump_data(data = tdata, file_path = logs_path, file_name = "logs_by_run_collapsed.csv")

#combine collapsed with results
data = pd.read_csv(os.path.join(logs_path, "logs_by_run_collapsed.csv"))
#change column names to avoid same name
data.columns = [col + "_collapsed" if col not in ["Player", "Function_index"] else col for col in data.columns]

results = pd.read_csv(os.path.join(logs_path, "results.csv"))
data = pd.merge(data, results, on=["Player","Function_index"])
lm.dump_data(data = data, file_path = logs_path, file_name = "extended_results.csv")

,Unnamed: 0_collapsed,forward_model_calls_collapsed,root_node_avg_reward_collapsed,nodes_count_collapsed,avg_leaf_node_depth_collapsed,max_leaf_node_depth_collapsed,min_leaf_node_depth_collapsed,std_leaf_node_depth_collapsed,leaf_node_count_collapsed,max_visits_collapsed,...,es_fitness_iterations,es_generations,es_semantics_l,es_semantics_u,use_semantics,parallel_evolution,no_terminal_no_parent,no_terminal_no_parent_terminals,re_evaluation,Path_origin


In [ ]:
#Get number of formulae that got stuck - not currently used
percentage_iterations = 0.126
data = pd.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
functions = data["Function_index"].unique()
mcts_variants = data["Player"].unique()
for mcts_variant in mcts_variants:
    for f in functions:
    
        if mcts_variant in data_order:
            tdata = data[(data["Function_index"]==f) & (data["Player"]==mcts_variant)]
            stuck_formulae = tdata[tdata["nodes_count"] < percentage_iterations * tdata["max_iterations"]]
            print("Function: {}, MCTS variant: {}, Stuck formulae: {}".format(f, mcts_variant, len(stuck_formulae)))


In [37]:
#CREATE DATASET for Statistical tests

#are the results normally distributed?
data = pd.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
data = data[data["Player"].isin(data_order)]

#create simplified dataset file
columns_of_interest = ["Player","Function_index","eval_of_max_visits_leaf_node","avg_leaf_node_depth", "std_leaf_node_depth"]
sdata = data[columns_of_interest]
lm.dump_data(data = sdata, file_path = logs_path, file_name = "logs_by_run_simplified.csv")

,Player,Function_index,eval_of_max_visits_leaf_node,avg_leaf_node_depth,std_leaf_node_depth


In [38]:
# Check normality across all combinations of algorithms and functions

# Load the data again to ensure we have a fresh start
data = pd.read_csv(os.path.join(logs_path, "logs_by_run_simplified.csv"))
normality_results = {}

# Get unique values for algorithms and function indices
algorithms = data['Player'].unique()
function_indices = data['Function_index'].unique()
#column_of_interest = "eval_of_max_visits_leaf_node"
column_of_interest = "avg_leaf_node_depth"

# Perform normality test for each combination
for algorithm in algorithms:
    for function_index in function_indices:
        # Filter the data for the current algorithm and function index
        subset = data[(data['Player'] == algorithm) & (data['Function_index'] == function_index)]
        
        # Perform the normaltest
        stat, p = scst.normaltest(subset[column_of_interest])
        
        # Store the p-value in the dictionary
        normality_results[(algorithm, function_index)] = p

# Now let's output the results and determine which combinations pass the normality test
# We'll use a common alpha level of 0.05 for the test
alpha = 0.05
normal_distributions = {k: v for k, v in normality_results.items() if v > alpha}

# Show which combinations are normally distributed
normal_distributions


{('EA_MCTS_its5000', 1): 0.08039499486689758,
 ('EA_MCTS_its5000', 2): 0.2609478268646763,
 ('MCTS_c1', 0): 0.9759083290214481,
 ('MCTS_c1', 2): 0.0680397782547685,
 ('MCTS_c1', 4): 0.4756858456442792,
 ('MCTS_c1_4142', 0): 0.4725874338473707,
 ('MCTS_c1_4142', 2): 0.06777378471523661,
 ('MCTS_c1_4142', 3): 0.2560773823678783,
 ('MCTS_c2', 0): 0.275178667603256,
 ('MCTS_c2', 2): 0.06735004660091082,
 ('MCTS_c2', 3): 0.40829730294566013,
 ('MCTS_c2', 4): 0.06152464171378998,
 ('MCTS_c3', 0): 0.38129394534461236,
 ('MCTS_c3', 1): 0.5196787063556998,
 ('MCTS_c3', 3): 0.09215605874740414,
 ('SIEA_MCTS_its5000', 2): 0.3897008094009191}

In [48]:
# To compare the results of EA_MCTS_its5000 and SIEA_MCTS_its5000, we will use a non-parametric test
# since the data is not normally distributed. The Mann-Whitney U test is appropriate for this purpose.

# Get unique values for algorithms and function indices
algorithms = data['Player'].unique()
function_indices = data['Function_index'].unique()
ea_based = ["EA_MCTS_its5000", "SIEA_MCTS_its5000"]
column_of_interest = "avg_leaf_node_depth"

for function_index in function_indices:
    for agent_i, agent1 in enumerate(algorithms):
        for agent2 in [a for a in algorithms[agent_i:] if a != agent1]:

            if agent2 not in ea_based:

                # Filter the data for the current function index
                subset = data[data['Function_index'] == function_index]
                
                # Perform the Mann-Whitney U test
                agent1_data = subset[subset['Player'] == agent1]['eval_of_max_visits_leaf_node']
                agent2_data = subset[subset['Player'] == agent2]['eval_of_max_visits_leaf_node']
                u_statistic, p_value = scst.mannwhitneyu(agent1_data, agent2_data, alternative='two-sided')

                #Perform the two-sample t-test with unequal variances (Welch's t-test)
                agent1_data = subset[subset['Player'] == agent1]['avg_leaf_node_depth']
                agent2_data = subset[subset['Player'] == agent2]['avg_leaf_node_depth']
                t_stat, tp_value = scst.ttest_ind(agent1_data, agent2_data, equal_var=False)
                
                # Display the results
                if tp_value > 0.05 and tp_value > 0.05:
                    print('SAME: Function index: {}, Agent 1: {}, Agent 2: {}, p-value: {}'.format(function_index, agent1, agent2, p_value))



SAME: Function index: 1, Agent 1: MCTS_c0_5, Agent 2: MCTS_c1, p-value: 1.860636663722952e-09
SAME: Function index: 3, Agent 1: MCTS_c0_5, Agent 2: MCTS_c1_4142, p-value: 4.965286113145066e-22


In [ ]:
#Evolution logs semantics usage
percentage_iterations = 0.126
data = pd.read_csv(os.path.join(logs_path, "evolution_logs.csv"))
print("original data len: ", len(data))
#functions = data["Function_index"].unique()
#mcts_variants = data["Player"].unique()
data["Player"] = [path_origin.split(os.sep)[3] for path_origin in data["Path_origin"]]              
tdata = data[data["Player"].isin(data_order)]
generations = len(tdata)
print("generations: ", generations)
semantics_used_data = tdata[tdata["semantics_used"]]
semantics_used_count = len(semantics_used_data)
print("semantics used \%: ", len(semantics_used_data)/len(tdata))
sem_rand = []
for chose_random in semantics_used_data["semantics_chose_randomly"]:
    if chose_random == 1 or chose_random == True:
        sem_rand.append(True)
    else:
        sem_rand.append(False)
semantics_used_data["chose_randomly"] = sem_rand
semantics_used_data_ok = semantics_used_data[semantics_used_data["chose_randomly"] == False]
print("semantics in U,L range \%: ", len(semantics_used_data_ok)/semantics_used_count) 
#print("MCTS variant: {}, Semantics usage: {}".format(mcts_variant, tdata["semantics_usage"].mean()))

In [ ]:
#Plot histograms

n_bins = 2**7#2**7
#logs_path = os.path.join("Outputs","FO_single_decision_final")
file_path_list = lm.find_log_files("tree_data.csv", logs_path)
#print(file_path_list)

#filter file_path_list
second_file_path_list = []
for fpl in file_path_list:
  agent_in_fpl = fpl.split(os.sep)[-2]
  #print("Agent in fpl:", agent_in_fpl)
  for agent_name in data_order:
    if agent_name == agent_in_fpl:
      second_file_path_list += [fpl]

file_path_list = second_file_path_list
#print("Filtered:",file_path_list)

#print(["\n" + f for f in file_path_list])
for function_index in range(5):
  data_list = []
  subplot_titles = []
  file_paths = []

  #Add mcts vanilla
  #"""
  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
      agent_name = file_path.split(os.sep)[-2]
      if "_c" in agent_name:
        data_list.append(pd.read_csv(file_path)) #mod to get interesting runs
        if "1_4142" in agent_name:
           agent_name= agent_name.replace("1_4142", "\u221A\u03052\u0305")
        if "0_5" in agent_name:
           agent_name= agent_name.replace("0_5", "0.5")
        agent_name = agent_name.replace("MCTS_", "MCTS ")
        agent_name = agent_name.replace(" c", " c = ")
        subplot_titles += [agent_name]
        file_paths += [file_path]
        paths_to_remove += [file_path]
  for path in paths_to_remove:
    file_path_list.remove(path)
  #"""

  #add ea
  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "EA_" in agent_name and not ("SIEA_" in agent_name or "SIEA2_" in agent_name):
      data_list.append(pd.read_csv(file_path))
      #agent_name= agent_name.replace("_its", " ")
      #agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]
  for path in paths_to_remove:
    file_path_list.remove(path)

  #add siea
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "SIEA_" in agent_name:
      data_list.append(pd.read_csv(file_path))
      #agent_name= agent_name.replace("_its", " ")
      #agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]

  #add ea2
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "SIEA2_" in agent_name:
      data_list.append(pd.read_csv(file_path))
      if agent_name in name_replaces.keys():
        agent_name = name_replaces[agent_name]
      #agent_name= agent_name.replace("_its", " ")
      #agent_name += " iterations"
      #if "SIEA2_MCTS" in agent_name:
      #  agent_name = agent_name.replace("SIEA2_MCTS", "EA_MCTS2")
      subplot_titles += [agent_name]
      file_paths += [file_path]
    
  if subplot_titles != [] and data_list != []:
    fop = fo.GameState(function_index=function_index)
    subplot_titles = [name_replaces[st] if st in name_replaces.keys() else st for st in subplot_titles]

    print("plotting")
    plot = eu.fo_tree_histogram_average(data_list,
                        fop.function, 
                        "F"+str(function_index+1) ,
                        divisions = 3,
                        n_buckets = n_bins, 
                        subplot_titles = [""] + subplot_titles, 
                        max_x_location=fop.max_location[0],
                        y_ref_value=None, height=800, width=800, extra_bottom_margin=0, legend_y = -0.075)
    plot.write_image(os.path.join(logs_path, "F" + str(function_index+1) + "_results"+ str(n_bins) + '.png'), width=800, height=800) 
    plot.show()
    #0,5,

In [6]:
#Plot interesting runs histograms. "the old one" for previous graph

n_bins = 2**7
logs_path = os.path.join("Outputs","FO_single_decision_final")
logs_path = os.path.join("Outputs","FOP")
file_path_list = lm.find_log_files("tree_data.csv", logs_path)
print(["\n" + f for f in file_path_list])
for function_index in [1]:#range(5):
  data_list = []
  logs_by_run_list = []
  subplot_titles = []
  file_paths = []

  #add ea
  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "EA_" in agent_name and not ("SIEA_" in agent_name or "SIEA2_" in agent_name):
      print("Found EA: ", agent_name, "in function ", function_index)
      data_list.append(pd.read_csv(file_path))
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]
      logs_by_run_list += [pd.read_csv(os.path.join(os.path.dirname(file_path), "logs_by_run.csv"))]
  for path in paths_to_remove:
    file_path_list.remove(path)
  """
  #add siea
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "SIEA_" in agent_name:
      data_list.append(pd.read_csv(file_path))  #mod to get interesting runs
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]

  #add ea2
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "SIEA2_" in agent_name:
      data_list.append(pd.read_csv(file_path))  #mod to get interesting runs
      if agent_name in name_replaces.keys():
        agent_name = name_replaces[agent_name]
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      #if "SIEA2_MCTS" in agent_name:
      #  agent_name = agent_name.replace("SIEA2_MCTS", "EA_MCTS2")
      subplot_titles += [agent_name]
      file_paths += [file_path]
  """
    
  if subplot_titles != [] and data_list != []:
    fop = fo.GameState(function_index=function_index)

    #print(len(data_list))
     #mod to get interesting runs
    #interesting_runs = [0,10,16,36,41,45,72,73,97] #the old one
    interesting_runs = range(10,20)
    #interesting_runs = [4, 9, 13, 40, 43, 48, 65, 84]
    #interesting_runs = [42,12,9,65]
    #interesting_runs = [
    interesting_data = []
    #interesting_titles = ["Tunnel","Even","Random", "Exploratory", "Opposite", "Emergent 1", "Emergent 2", "Emergent 3", "UCB1 c=0.25"]
    #interesting_titles = ["Behaviour type " + str(i+1) for i in range(len(interesting_runs))] #the old one
    interesting_titles = []#[str(ir) for ir in interesting_runs]
    """ the old one
    for j in range(10):
      new_data_list = []
      subplot_titles = []
      dat = data_list[1] if len(data_list) > 1 else data_list[0]
      
      for i in range(j*10,(j+1)*10):
        tdat = dat.loc[dat["run"] == i]
        new_data_list = new_data_list + [tdat]
        if i==40:
          my_dat = tdat
        if i in interesting_runs:
          interesting_data += [tdat]
        subplot_titles += ["Run " + str(i)]
      subdata_list = new_data_list
    """
    for i in interesting_runs: #the new one
      tdat = data_list[0].loc[data_list[0]["run"] == i]
      interesting_data += [tdat]
      #subplot_titles += ["Run " + str(i)]
      evolved_formula = logs_by_run_list[0].loc[logs_by_run_list[0]["run"] == i]["evolved_formula"]
      #print(evolved_formula)
      interesting_titles += [str(i) + ": " + evolved_formula.values[0] + "nodes" + str(len(tdat))]

    
    print("plotting")
    plot = eu.fo_tree_histogram_average(interesting_data,
                        fop.function, 
                        "F"+str(function_index+1) ,
                        divisions = 3,
                        n_buckets = n_bins, 
                        subplot_titles = [""] + interesting_titles,
                        max_x_location=fop.max_location[0],
                        y_ref_value=None,
                        #divnames = ["Evolutionary process","Evolved formula"], div_legend_name="Node expansion stage",
                        min_x_location=0.632,
                        #chunks = [0, 2430, 5000]
                        )
    if function_index == 1:
      #plot.write_image(os.path.join(logs_path, "Interesting_runs.png"))#"F" + str(function_index+1) + "_results"+ str(n_bins) + '.png'))#, width=800, height=1000) 
    #plot.write_image(os.path.join(logs_path, "F" + str(function_index+1) + "_results"+ str(n_bins) + '_all_agents.png'), width=800, height=1000) 
      plot.show()
    #0,5,

['\nOutputs\\FOP\\Function_0\\EA_MCTS_its5000\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\MCTS_c0_5\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\MCTS_c1\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\MCTS_c1_4142\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\MCTS_c2\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\MCTS_c3\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\SIEA2_MCTS_discard\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\SIEA2_MCTS_its5000\\tree_data.csv', '\nOutputs\\FOP\\Function_0\\SIEA_MCTS_its5000\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\EA_MCTS_its5000\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\MCTS_c0_5\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\MCTS_c1\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\MCTS_c1_4142\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\MCTS_c2\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\MCTS_c3\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\SIEA2_MCTS_discard\\tree_data.csv', '\nOutputs\\FOP\\Function_1\\SIEA2_MCTS_its5000\\tree_data.csv

In [24]:
import plotly.graph_objects as go
import numpy as np

# Define the function
def f(x, z):
    return x + 2 * z
def ucb(reward, n, c=1.4142, N=10):
    """
    Calculate the Upper Confidence Bound (UCB).

    :param reward: The observed reward.
    :param n: The number of times the option has been chosen.
    :param c: Exploration parameter (default is approximately sqrt(2)).
    :return: The UCB value.
    """
    return reward + c * math.sqrt(math.log(n) / N)

# Define the x and z ranges
x_range = np.linspace(0, 1, 100)
z_range = np.linspace(1, 100, 100)

# Initialize arrays for X, Z, and Y
X, Z, Y = [], [], []

# Iterate through the ranges and calculate Y
for x in x_range:
    for z in z_range:
        X.append(x)
        Z.append(z)
        Y.append(ucb(x, z))

# Convert lists to numpy arrays for plotting
X = np.array(X)
Z = np.array(Z)
Y = np.array(Y)

# Create the contour plot using Plotly
fig = go.Figure(data=go.Contour(
    x=X, 
    y=Z, 
    z=Y, 
    colorscale='Viridis',
    contours=dict(
        coloring='heatmap'
    )
))

# Update layout
fig.update_layout(
    title='Contour plot of y = x + 2z with color gradient',
    xaxis_title='x',
    yaxis_title='z',
    coloraxis_colorbar=dict(title='Values of y')
)

#change figure size
fig.update_layout(margin=dict(l=50, r=10, t=30, b=30)
                        ,width=600
                        ,height=500
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        #,boxmode='group'
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,legend=dict(
                            title = "Function grid"
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.075
                            #,xanchor="center"
                            #,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            ) 
                        )

fig.show()

In [18]:
#CP: FOP boxplot best reward path

#define variables
#logs_path = os.path.join("Outputs","FO_single_decision_4")
titles = ["Highest reward node-based result by MCTS variant", "Most visited node-based result by MCTS variant"]
boxplot_data = ["eval_of_max_reward_leaf_node", "eval_of_max_visits_leaf_node"]
x_axes_names = ["Highest reward node", "Most visited node"]
#filter_out = {"player_name":["SIEA2_MCTS_its2570", "SIEA2_MCTS_its5000"]}
assert len(titles) == len(boxplot_data), "Titles and boxplot_data must have the same length"

for i in [1]:#range(len(titles)):
    logs = lm.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
    title_text = titles[i]
    y_columns = [boxplot_data[i]]
    plot_separation_column = "Function_index"
    x_column = "player_name"
    #sort dataset according to order:
    logs["order"] = pd.Categorical(logs["player_name"], data_order)
    logs = logs.sort_values("order")
    logs = logs.dropna(subset=["order"])

    #change SIEA2_its2570 for EA2_its2570
    for name_replace in name_replaces.keys():
         logs["player_name"] = logs["player_name"].replace(name_replace, name_replaces[name_replace])
    #logs["player_name"] = logs["player_name"].replace("SIEA2_MCTS_its2570", "EA_MCTS2_its2570")
    #logs["player_name"] = logs["player_name"].replace("SIEA2_MCTS_its5000", "EA_MCTS2_its5000")

    #filters
    #logs = logs[logs["player_name"] != "EA_MCTS2_its2570"]
    #logs = logs[logs["player_name"] != "EA_MCTS2_its5000"]
    #logs = logs[logs["player_name"] != "EA_MCTS2_its7430"]
    

    #Calculate uniques
    plot_identifiers = logs[plot_separation_column].unique()
    #sort plot identifiers
    plot_identifiers = sorted(plot_identifiers, key=lambda x: x)
    #box_group_identifiers = logs[box_group_column].unique()
    print("plot_identifiers", plot_identifiers)
    #print("box_group_identifiers", box_group_identifiers)

    #Create figure
    n_rows = len(y_columns)
    fig = make_subplots(rows=n_rows
                            ,cols=len(plot_identifiers)
                            ,shared_xaxes=True
                            ,shared_yaxes=True
                            #,vertical_spacing=0.04
                            #,row_heights=[1/n_rows for _ in range(n_rows)]
                            ,subplot_titles = ["F" + str(i+1) for i in plot_identifiers]*n_rows
                            #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                            ,x_title="Result"
                            #,y_title='Max reward path'
                            #,print_grid=True
                            )

    for row, y_column in enumerate(y_columns):
        for col, plot_identifier in enumerate(plot_identifiers):
            #for group_id, box_group_identifier in enumerate(box_group_identifiers):
                data = logs[logs[plot_separation_column] == plot_identifier]
                #data = data[data[box_group_column] == box_group_identifier]
            
                fig.add_trace(go.Box(x=data[y_column]#[0 for _ in range(len(data))]#data[x_column]
                                    ,y=data[x_column]
                                    #,name=str(box_group_identifier)
                                    , width=0.9
                                    ,showlegend=False#row==0
                                    #,legendgroup=str(box_group_identifier)
                                    ,orientation = "h"
                                    , marker={"color":colors[1]}
                                    )
                                ,row=row + 1
                                ,col=col + 1)

    fig.update_layout(margin=dict(l=70, r=10, t=30, b=50)
                            ,width=1000
                            ,height=400
                            ,plot_bgcolor='rgba(0,0,0,0)'
                            #,plot_bgcolor="lightgray"
                            #,boxmode='group'
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,legend=dict(
                                title = "Rewards by MCTS variant"
                                ,orientation="h"
                                ,yanchor="top"
                                ,y=-0.075
                                ,xanchor="center"
                                ,x=0.4
                                ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                                ,font = dict(family = "Arial", size = 14, color = "black")
                                ,bordercolor="Black"
                                ,borderwidth=2
                                ,itemsizing='trace'
                                ,itemwidth = 30
                                ) 
                            )
    fig.update_layout(boxmode='group')
    fig.update_xaxes(showline=True
                            , linewidth=2
                            , linecolor='black'
                            , mirror=True
                            ,showgrid=True
                            , gridcolor="black"
                            , gridwidth=0.15
                            #, nticks=5
                            ,tickmode = 'linear'
                            ,tick0 = 0
                            , dtick=0.25
                            ,range= [0.5,1]
                            )

    fig.update_yaxes(showline=True
                        ,mirror=True
                        , linewidth=2
                        , linecolor='black'
                        )
    #give title to the plot
    #fig.update_layout(title_text=title_text)

    fig.write_image(os.path.join(logs_path, title_text.replace(" ", "_").replace("-","_")+"_siea1.png"))
    fig.show()


plot_identifiers [0, 1, 2, 3, 4]


In [8]:
#CP: FOP boxplot best reward path

#define variables
#logs_path = os.path.join("Outputs","FO_single_decision_4")
logs = lm.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
y_columns = ["eval_of_max_reward_leaf_node", "eval_of_max_visits_leaf_node"] #this controls the groups
group_names = ["Highest reward node-based reward", "Most visited node-based reward"]
xaxis_name = "Reward"
title_text = "Optimal play reward by MCTS variant"
plot_separation_column = "Function_index"
#box_group_column = "player_name"
x_column = "player_name"#"current_iterations"#"pg_game_turn"
#sort dataset according to order:
logs["player_name"] = pd.Categorical(logs["player_name"], data_order)
logs = logs.sort_values("player_name")

#change SIEA2_its2570 for EA2_its2570
for name_replace in name_replaces.keys():
         logs["player_name"] = logs["player_name"].replace(name_replace, name_replaces[name_replace])

#filters
logs = logs[logs["player_name"] != "EA_MCTS2_its2570"]
logs = logs[logs["player_name"] != "EA_MCTS2_its5000"]

#Calculate uniques
plot_identifiers = logs[plot_separation_column].unique()
#sort plot identifiers
plot_identifiers = sorted(plot_identifiers, key=lambda x: x)
#box_group_identifiers = logs[box_group_column].unique()
colors = ["#5B8C5A","#56638A","#EC7316","#e4c9db","#e4deb8","#e265ef","#c27dd1","#d52e87","#a8d0b9","#ad8dfd"]
print("plot_identifiers", plot_identifiers)

#Create figure
n_rows = 1
fig = make_subplots(rows=n_rows
                        ,cols=len(plot_identifiers)
                        ,shared_xaxes=True
                        ,shared_yaxes=True
                        #,vertical_spacing=0.04
                        #,row_heights=[1/n_rows for _ in range(n_rows)]
                        ,subplot_titles = ["Function " + str(i+1) for i in plot_identifiers]*n_rows
                        #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                        ,x_title="Reward"
                        #,y_title='Max reward path'
                        #,print_grid=True
                        )

for y_i, y_column in enumerate(y_columns):
    for col, plot_identifier in enumerate(plot_identifiers):
        #for group_id, box_group_identifier in enumerate(box_group_identifiers):
            data = logs[logs[plot_separation_column] == plot_identifier]
            #data = data[data[box_group_column] == box_group_identifier]
        
                
            fig.add_trace(go.Box(x=data[y_column]#[0 for _ in range(len(data))]#data[x_column]
                                ,y=data[x_column]
                                ,name=group_names[y_i]
                                ,whiskerwidth=1
                                #,marker_size=5
                                #,line_width=1
                                , width=0.8
                                ,showlegend=col==0
                                ,opacity=0.8
                                #,legendgroup=str(box_group_identifier)
                                ,orientation = "h"
                                , marker={"color":colors[y_i]}
                                )
                            ,row=1
                            ,col=col + 1)

fig.update_layout(margin=dict(l=70, r=10, t=70, b=100)
                        ,width=1000
                        ,height=400
                        ,plot_bgcolor='rgba(0,0,0,0)'
                        #,plot_bgcolor="lightgray"
                        #,boxmode='group'
                        ,font = dict(family = "Arial", size = 14, color = "black")
                        ,legend=dict(
                            title = "Rewards by MCTS variant"
                            ,orientation="h"
                            ,yanchor="top"
                            ,y=-0.250
                            ,xanchor="center"
                            ,x=0.4
                            ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,bordercolor="Black"
                            ,borderwidth=2
                            ,itemsizing='trace'
                            ,itemwidth = 30
                            ) 
                        )
fig.update_xaxes(showline=True
                        , linewidth=2
                        , linecolor='black'
                        , mirror=True
                        ,showgrid=True
                        , gridcolor="black"
                        , gridwidth=0.15
                        #, nticks=5
                        ,tickmode = 'linear'
                        ,tick0 = 0
                        , dtick=0.25
                        ,range= [0,1]
                        )
fig.update_yaxes(showline=True
                    ,mirror=True
                    , linewidth=2
                    , linecolor='black'
                    )
#give title to the plot
fig.update_layout(title_text=title_text)
fig.update_layout(boxmode='group'
                  #,width=0.4
                #,boxgroupgap = 0
                #,boxgap = 0.1
                )
#fig.write_image(os.path.join(logs_path, title_text.replace(" ", "_").replace("-","_")+".png"))
fig.show()

plot_identifiers [0, 1, 2, 3, 4]


In [20]:
#LEAF NODES DEPTHS

#CP: FOP boxplot best reward path
#define variables
#logs_path = os.path.join("Outputs","FO_single_decision_4")
titles = ["Average leaf node depth by MCTS variant", "Average leaf node count by MCTS variant", "Average expanded nodes"]
boxplot_data = ["avg_leaf_node_depth", "leaf_node_count", "nodes_count"]
xaxis_name = ["Average leaf nodes depth", "Average leaf nodes count", "Average expanded nodes count"]
dticks = [2, 1000, 1000]
x_ranges = [[None,None], [0,2500], [0,5100]]
assert len(titles) == len(boxplot_data), "Titles and boxplot_data must have the same length"

for i in range(len(titles)):
    logs = lm.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
    title_text = titles[i]
    y_columns = [boxplot_data[i]]

    plot_separation_column = "Function_index"
    x_column = "player_name"#"current_iterations"#"pg_game_turn"
    #sort dataset according to order:
    logs["order"] = pd.Categorical(logs["player_name"], data_order)
    logs = logs.sort_values("order")
    logs = logs.dropna(subset=["order"])

    #change SIEA2_its2570 for EA2_its2570
    for name_replace in name_replaces.keys():
         logs["player_name"] = logs["player_name"].replace(name_replace, name_replaces[name_replace])

    #filters
    #logs = logs[logs["player_name"] != "EA_MCTS2_its2570"]
    #logs = logs[logs["player_name"] != "EA_MCTS2_its5000"]
    #logs = logs[logs["player_name"] != "EA_MCTS2_its7430"]

    #Calculate uniques
    plot_identifiers = logs[plot_separation_column].unique()
    #sort plot identifiers
    plot_identifiers = sorted(plot_identifiers, key=lambda x: x)
    #box_group_identifiers = logs[box_group_column].unique()
    print("plot_identifiers", plot_identifiers)
    #print("box_group_identifiers", box_group_identifiers)

    #Create figure
    n_rows = len(y_columns)
    
    fig = make_subplots(rows=n_rows
                            ,cols=len(plot_identifiers)
                            ,shared_xaxes=True
                            ,shared_yaxes=True
                            #,vertical_spacing=0.04
                            #,row_heights=[1/n_rows for _ in range(n_rows)]
                            ,subplot_titles = ["F" + str(i+1) for i in plot_identifiers]*n_rows
                            #, specs=[[{"secondary_y": True}] for _ in range(n_plots+1)]
                            ,x_title=xaxis_name[i]
                            #,y_title='Max reward path'
                            #,print_grid=True
                            )

    for row, y_column in enumerate(y_columns):
        for col, plot_identifier in enumerate(plot_identifiers):
            #for group_id, box_group_identifier in enumerate(box_group_identifiers):
                data = logs[logs[plot_separation_column] == plot_identifier]
                #data = data[data[box_group_column] == box_group_identifier]
            
                #print(data[x_column].unique())
                fig.add_trace(go.Box(x=data[y_column]#[0 for _ in range(len(data))]#data[x_column]
                                    ,y=data[x_column]
                                    #,name=str(box_group_identifier)
                                    , width=0.9
                                    ,showlegend=False#row==0
                                    #,legendgroup=str(box_group_identifier)
                                    ,orientation = "h"
                                    , marker={"color":colors[1]}
                                    )
                                ,row=row + 1
                                ,col=col + 1)

    fig.update_layout(margin=dict(l=70, r=10, t=30, b=50)
                            ,width=1000
                            ,height=400
                            ,plot_bgcolor='rgba(0,0,0,0)'
                            #,plot_bgcolor="lightgray"
                            #,boxmode='group'
                            ,font = dict(family = "Arial", size = 14, color = "black")
                            ,legend=dict(
                                title = "Rewards by MCTS variant"
                                ,orientation="h"
                                ,yanchor="top"
                                ,y=-0.075
                                ,xanchor="center"
                                ,x=0.4
                                ,bgcolor="white"#"lightgray"#"rgba(200, 255, 255, 0.8)"
                                ,font = dict(family = "Arial", size = 14, color = "black")
                                ,bordercolor="Black"
                                ,borderwidth=2
                                ,itemsizing='trace'
                                ,itemwidth = 30
                                ) 
                            )
    fig.update_layout(boxmode='group')
    fig.update_xaxes(showline=True
                            , linewidth=2
                            , linecolor='black'
                            , mirror=True
                            ,showgrid=True
                            , gridcolor="black"
                            , gridwidth=0.15
                            #, nticks=5
                            #,tickmode = 'linear'
                            , tick0 = 0
                            , dtick=dticks[i]
                            )
    if x_ranges[i] != [None,None]:
        fig.update_xaxes(range=x_ranges[i])

    fig.update_yaxes(showline=True
                        ,mirror=True
                        , linewidth=2
                        , linecolor='black'
                        )
    #give title to the plot
    #fig.update_layout(title_text=title_text)

    fig.write_image(os.path.join(logs_path, title_text.replace(" ", "_").replace("-","_")+"siea1.png"))
    fig.show()


plot_identifiers [0, 1, 2, 3, 4]


plot_identifiers [0, 1, 2, 3, 4]


plot_identifiers [0, 1, 2, 3, 4]


In [1]:
#LUDO

class LudoArrow():
    def __init__(self,  end_index=None, player_house=None):
        self.player_house = player_house #The player whose house this is
        self.end_index = end_index

class LudoSquare():
    def __init__(self, index, is_safe=False, arrow=None, content=None, next_square_index=None, player_house=None, is_victory_path=False):
        self.index = index
        self.is_safe = is_safe
        self.content = content #A list of pieces
        self.next_square_index = next_square_index #The index of the next square
        self.player_house = player_house #The player whose house this sends to
        self.is_victory_path = is_victory_path #Whether this is a victory path square

class LudoPiece():
    def __init__(self, index, player):
        self.index = index
        self.player = player
        
class GameState():
    def __init__(self, n_players, pieces_per_player=4):
        self.players = ["red", "green", "blue", "yellow"]
        self.n_players = n_players
        self.pieces_per_player = pieces_per_player
    def set_initial_state(self):
        self.board = {}
        self.players_houses = {12:"red", 25:"green", 38:"blue", 51:"yellow"}
        self.players_starting_index = {"red":15, "green":28, "blue":41, "yellow":2}
        for i in range(52):
            if i in [2, 10, 15, 23, 28, 36, 41, 49]: #safe squares
                square = LudoSquare(i, is_safe=True)
            elif i in [6, 19, 32, 45]: #squares with immediate arrows
                arrow = LudoArrow(end_index=i+1)
                square = LudoSquare(i, arrow=arrow)
            elif i in [10, 23, 36, 49]: #squares with arrows to the player houses
                arrow = LudoArrow(end_index=53, player_house=i)
                square = LudoSquare(i, arrow=arrow)
            elif i in [13, 26, 39, 52]:
                square = LudoSquare(i, player_house=self.players_houses[i])







In [ ]:
#LORCANA API
import requests
import json
import pandas as pd

names_request = requests.get('https://api.lorcana-api.com/lists/names')
if names_request.status_code == 200:
    response_string = str(names_request.content).split("\\n")
print("Got ", str(len(response_string)), " names from the API")

names = []
card_dataset = []
failed_cards = []
for raw_name in response_string:
    card_data = None
    attempts = 0
    while attempts < 5:
        if "," in raw_name: #Filtering for missing cards
            name = raw_name.replace("\"", "")
            name = name.replace("\\", "")
            name = name.replace(" ", "")
            name = name.replace(",", "")

            full_string = 'https://api.lorcana-api.com/strict/' + name
            print("Trying string: ", full_string)
            #print(full_string)
            card_data = requests.get(full_string)
            if card_data.status_code == 200:
                print(card_data.content)
                break
            else:
                print('Request failed with status code:', card_data.status_code)
                card_data = None
                attempts += 1
        else:
            print("Skipping: ", raw_name)
            break
        
    #Save the data in a dictionary
    if card_data is not None:
        card_dict = json.loads(card_data.content)
        card_dataset.append(card_dict)
    else:
        failed_cards.append(raw_name)
        print("Failed to get data for: ", raw_name)
    
#Merge all the dicionaries in a single dataframe
df = pd.DataFrame()
for card_dict in card_dataset:
    #card_df = pd.DataFrame(card_dict, index = [0])
    card_df = pd.DataFrame()
    for key,value in card_dict.items():
        print(key, value)
        card_df[key] = [value]
    df = pd.concat([df, card_df], axis = 0)
    #Save df to csv
    df.to_csv("card_data.csv", index = False)